# Machine Learning Series Seminar: Reinforcement Learning Module

## 1. Introduction: Reinforcement Learning

[Reinforcement Learning](http://en.wikipedia.org/wiki/Reinforcement_learning) is concerned with finding the optimal <i>action</i> to take for any given <i>state</i> an agent may encounter in an <i>environment</i> in order to maximize some <i>reward</i> value.  These state-action mappings are called <i>policies</i>.  The key difference from supervised learning is that for each state we are not told the correct action, instead we are only given the cumulative reward after some evaluation period (delayed reward).  For example, consider a learning to play soccer.  The reward signal is: Did we win?, however there many actions taken by players during the game that could have contributed to winning, losing, or not at all. How much did scoring that goal, or making that pass, or moving to that location contribute to winning?  See [Sutton & Barto](http://webdocs.cs.ualberta.ca/~sutton/book/ebook/the-book.html) for a full introduction to reinforcement learning.

For very small state and action spaces, these policies can all be evaluated through <i>brute force</i>; however, for almost any interesting domain, the state and action space is going to be so large that the cost of testing all possible policies will be prohibitively high.  Thus reinforcement learning focuses on creating algorithms to find these policies without resorting to exhaustive search.

The first approach to be explored in this module is Temporal Difference Learning (TD-Learning), a variant of <i>value-function learning</i>.  Briefly, such approaches focus on learning the reward structure of the domain, that is, a policy is continually evaluated against the domain to determine what rewards particular state-action mappings receive and then the policy is updated to maximize the cumulative reward as determined by the learned reward structure.  Such approaches are often based upon [Markov Decision Process](http://en.wikipedia.org/wiki/Markov_decision_process) theory and rely on the related underlying assumptions, such as the [Markov property](http://en.wikipedia.org/wiki/Markov_property).

The second approach in this module is a type of <i>direct policy search</i>.  In policy search, a stochastic optimization approach is applied directly on policies in order to find the optimal ones.  These approaches fall into either gradient-based (optimize the policy based upon gradient information extracted through evaluations) or gradient-free. In particular, this module will examine the gradient-free approach [Evolutionary Computation](http://en.wikipedia.org/wiki/Evolutionary_computation), inspired by the theory of natural selection. 

The next section discusses the TD-Learning approach.

## 2. Temporal Difference (TD) Learning

A key characteristic of value-function learning approaches, such as TD-Learning, is that they naturally consider the entire learning problem in an online decision-making framework.  That is, this form of reinforcement learning is concerned with how the learning agent interacts with the environment, what the learning agent can measure from the environment, what is the ultimate goal of the learning agent, and how to deal with uncertainty. Reinforcement learning of this type closely resembles learning observed in nature wherein it is rare, if even possible, to have a teacher available who can provide the <i>right</i> actions for a learner to follow in <i>every</i> possible state. One of the unique challenges of this learning is the balance between <i>exploration</i> and <i>exploitation</i>, that is, the agent has to <i>exploit</i> what it already knows in order to obtain a reward, but it also has to <i>explore</i> in order to find actions that yield better rewards. 

To illustrate the need to balance exploration and exploitation, consider a gambler presented with a number of slot machines.  Each slot machine has a different parameters for their payout rates that are unknown to the gambler.  Most of the machines cause the gambler to lose money, some let the gamble break even, and a few allow the gambler to win.  If the gambler plays all the machines, he will lose on average.  If the gambler focuses only on the first machine that seems to payout, the optimal machine may be ignored.  Thus the gambler needs to expend some time to try different machines in order to model their rewards, but must also accumulate as much reward as possible with the time/money they have.  This problem is known as the <i>Multi-armed Bandit</i> problem. 

TD-Learning is described in the framework of Markov Decision Processes (MDP), which is a strong theoretical model for decision making in situations where the transition of the environment is stochastic. An agent with full knowledge of the MDP can seek for an optimal policy that maximizes some function of its expected cumulative reward. If the MDP is unknown, then an agent learns to take optimal actions through interactions with the environment alone.  MDPs are described next.

### 2.1. Markov Decision Processes (MDPs)

Sequential decision making problems are often cast as a Markov Decision Process (MDP). An MDP is comprised by a tuple $(\mathcal{S},\mathcal{A},\mathcal{P},\mathcal{R})$ where $\mathcal{S}$ denotes the set of states, $\mathcal{A}$ the set of actions, $\mathcal{P}$ a set of transition probabilities $P_{s,s'}^a:=Pr(s'|s,a)$ each representing the probability of transitioning from $s\in\mathcal{S}$ to $s'\in\mathcal{S}$ after taking action $a\in\mathcal{A}$, and $\mathcal{R}$ a set of rewards $r_{s,s'}^a$ each representing the one-step reward obtained when transitioning from $s$ to $s'$ after taking $a$. 

Note that MDPs are underpinned by several assumptions.  On such property is that the transition probabilities for a particular action rely only on the current state, i.e.:

$$
Pr(s_{t+1}=s'~|~s_t=s,a_t=a)=Pr(s_{t+1}=s'~|~s_t=s,a_t=a,s_{t-1}=s,a_{t-1}=a,\ldots,s_0=s,a_0=a)
$$

That is, the system is <i>memoryless</i>, also known as the <i>Markov property</i>, thus an agent can predict the next state and expected reward of the MDP based on the current state $s$ and action $a$ alone. Hence, the name Markov Decision Process.  Another assumption is that the world is <i>stationary</i>, that is, the system model does not change.  Finally, for MDPs a common assumption is full observability, meaning the entire state of the world can be observed at each time step.  A looser restriction on this assumption results in what is known as a Partially Observable Markov Decision Process (POMDP), which represent a great deal of real-world problems.

A trajectory is a sequence $\{s_0,a_0,r_0,s_1,a_1,r_1,\ldots\}$, where $r_t:=r^{a_t}_{s_t,s_{t+1}}$ and $s_t$ ($a_t$) denotes the state (action) observed (taken) at time $t$. Each action $a_t\in\mathcal{A}$ in the trajectory is chosen according to a policy $\pi:\mathcal{S}\rightarrow \mathcal{A}$, which maps each state to a given action. Although this module focuses on deterministic policies, it is important to remark that probabilistic policies can also be developed where the policy yields a probability distribution over $\mathcal{A}$. 

Given a policy $\pi$, the state-action value function $Q^{\pi}(s,a)$ for each state-action pair is 

$$
Q^{\pi}(s,a):=\mathbb{E}_{\pi}\left[\sum_{t=0}^{\infty}\gamma^tr_t ~|~s_0=s,a_0=a\right]
$$

where the discount factor $\gamma\in(0,1)$ down weights the value of future rewards, and $\mathbb{E}_{\pi}$ denotes the expected value with respect to the state given that the agent follows policy $\pi$. The state-action value function quantifies how good it is for the learning agent to take action $a$ when in state $s$. Similarly, the state value function for a given $\pi$ is

\begin{align}
V^{\pi}(s)&:=\mathbb{E}_{\pi}\left[\sum_{t=0}^{\infty}\gamma^tr_t ~|~s_0=s\right]\\
&=Q^{\pi}(s,\pi(s))
\end{align}

which quantifies how good it is for the agent to be in state $s$. A subtle difference between $V^{\pi}(s)$ and $Q^{\pi}(s,a)$ is that $\forall t\geq0$, $a_t$ is chosen according to $\pi$ in the former, and $a_t$ is chosen accordingly to $\pi$ $\forall t>0$ with $a_0$ fixed as $a_0=a$ in the latter. The state value function can be written recursively as $V^{\pi}(s)=\mathbb{E}_{\pi}\left[r_{s,s'}^{\pi(s)}+\gamma V^{\pi}(s')\right]$. This expression relates the value of being at a given state $s$ to that of being at successor states $s'\in\mathcal{S}$. 

The planning problem for an MDP is, given the tuple $(\mathcal{S},\mathcal{A},\mathcal{P},\mathcal{R})$ find an optimal policy $\pi^*(s)$ that maximizes the expected cumulative discounted rewards

$$
\pi^*(s)=\mathop{\arg\max}_{\pi}~~\mathbb{E}_{\pi}\left[\sum_{t=0}^{\infty}\gamma^tr_t ~|~s_0=s\right].
$$

Note that $\pi^*$ depends on a given initial state $s$. The optimal value function $V^*(s):=V^{\pi^*}(s)$ satisfies the Bellman optimality equation

\begin{align}
V^*(s) &=\max_{a\in\mathcal{A}}~\mathbb{E}_{\pi^*}\left[r_{s,s'}^a+\gamma V^*(s')\right]\\
&=\max_{a\in\mathcal{A}}~\sum_{s'\in\mathcal{S}}P_{s,s'}^a\left[r_{s,s'}^a+\gamma V^*(s')\right].
\end{align}
 
 Given a well-defined MDP, it is possible to directly solve for the optimal policy, as described next.

<b>Key Terms</b>
   <ul><li><b>Environment</b>:        World with which a learning agent interacts</li>
       <li><b>State ($s$)</b>:        Observations that define the environment for the agent</li>
       <li><b>State-Space ($S$)</b>:  All possible values of the state</li>
       <li><b>Action ($a$)</b>:       Decision by agent that interacts with the environment</li>
       <li><b>Action-Space ($A$)</b>: All possible actions the agent may make</li>
       <li><b>State Transition:</b>   Moving from one state to another state given an action</li>
       <li><b>Policy ($\pi$)</b>:     Mapping of states to actions ($\pi(s)=a$)</li>
       <li><b>Reward ($r$)</b>:       Benefit received from a particular state</li>
   <ul>

### 2.2. Directly Solving the MDP

Dynamic programming is an iterative approach for solving sequential decision-making problems. It rests on the principle of optimality which states that an optimal policy has the property that whatever the initial state and initial decision are, the remaining decisions must constitute an optimal policy with regard to the state resulting from the first decision. An instantiation of dynamic programming can be used to solve the planning problem for an MDP by finding the optimal state value function and its corresponding optimal policy. The state value function $V^{\pi}(s)$ can be written recursively as a function of the underlying policy $\pi$ as

\begin{align}
V^{\pi}(s) &=\sum_{s'\in\mathcal{S}}P_{s,s'}^{\pi(s)}\left[r_{s,s'}^{\pi(s)}+\gamma V^{\pi}(s')\right],~~\forall s\in\mathcal{S}.
\end{align}

Note that the latter set of equations is different from the Bellman optimality equations in that it is policy dependent.

When $\mathcal{S}:=\{s_1,\ldots,s_{|\mathcal{S}|}\}$ is finite, the state value functions can be calculated in closed form by solving $|\mathcal{S}|$ linear equations, one for each state of the MDP. Let $\mathbf{P}\in\mathbb{R}^{|\mathcal{S}|\times |\mathcal{S}|}$ denote the matrix of transition probabilities where its $(i,j)$ entry is given by $\mathbf{P}_{i,j}:=P_{s_i,s_j}^{\pi(s_i)}$, $\mathbf{v}^{\pi}\in\mathbb{R}^{|\mathcal{S}|}$ the vector of value functions of the policy $\pi$, and $\mathbf{r}\in\mathbb{R}^{|\mathcal{S}|}$ the vector of expected rewards with entries $\mathbf{r}_i:=\sum_{j=1}^{|\mathcal{S}|}P_{s_i,s_j}^{\pi(s_i)}r_{s_i,s_j}^{\pi(s_i)}$. Then, the value function can be readily written as $\mathbf{v}=\mathbf{r}+\gamma\mathbf{Pv}$ and be written in closed form as

$$
\hat{\mathbf{v}}=(\mathbf{I}-\gamma\mathbf{P})^{-1}\mathbf{r}
$$

where $\mathbf{I}$ is a $|\mathcal{S}|\times |\mathcal{S}|$ identity matrix and the inverse $(\mathbf{I}-\gamma\mathbf{P})^{-1}$ always exists. However, computing $\hat{\mathbf{v}}$ directly can be computational prohibited, specially for large $|\mathcal{S}|$. Instead, a fixed-point iteration can be used to iteratively obtain $\hat{\mathbf{v}}$ as

$$
\mathbf{v}^{(\tau+1)}\leftarrow \mathbf{r}+\gamma\mathbf{P}\mathbf{v}^{(\tau)}
$$

where $\tau$ denotes an iteration index. This fixed-point iteration is guaranteed to converge to $\hat{\mathbf{v}}$ as $\tau\rightarrow\infty$. The policy improvement step is done by using the Bellman optimality equations and $\hat{\mathbf{v}}$ to select an action greedily as

$$
\pi(s_i) =\mathop{\arg\max}_{a\in\mathcal{A}}~\sum_{j=1}^{|\mathcal{S}|}P_{s_i,s_j}^a\left[r_{s_i,s_j}^a+\gamma \hat{\mathbf{v}}_j\right], ~~\forall s_i\in\mathcal{S}.
$$

Performing iteratively the policy evaluation and policy improvement steps yields the <i>Policy Iteration</i> algorithm. Although Policy Iteration is guaranteed to converge to the optimal solution of the Bellman optimality equations in polynomial time, it is not scalable due to its elevated computational and storage complexity given by $O(|\mathcal{S}|^3)$ and $O(|\mathcal{S}|^2)$, respectively.

Convergence to the optimal policy can still be guaranteed if the policy improvement step is performed before obtaining $\hat{\mathbf{v}}$ exactly as long as ${\mathbf{v}}$ is improved after each policy evaluation step. Thus, the policy improvement step whould use the best possible action for evaluation rather than a fixed policy. The resulting algorithm, known as <i>Value Iteration</i>, comprises the following updates

\begin{align}
\tilde{\mathbf{v}}_i &=\max_{a\in\mathcal{A}}~\sum_{j=1}^{|\mathcal{S}|}P_{s_i,s_j}^a\left[r_{s_i,s_j}^a+\gamma \tilde{\mathbf{v}}_j\right], ~~\forall s_i\in\mathcal{S}\\
\pi(s_i) &=\mathop{\arg\max}_{a\in\mathcal{A}}~\sum_{j=1}^{|\mathcal{S}|}P_{s_i,s_j}^a\left[r_{s_i,s_j}^a+\gamma \tilde{\mathbf{v}}_j\right], ~~\forall s_i\in\mathcal{S}.
\end{align}

Value Iteration improves the policy much more frequently and reduces the per-iteration complexity of the Policy Iteration algorithm. In practice Value Iteration often requires less time to find an optimal policy when compared to the Policy Iteration algorithm.

To demonstrate how Value Iteration solves these MDP problems, we introduce a grid world domain below.  In this domain, the agent lives in a 3 $\times$ 4 grid, with the state of the agent defined as the $(x,y)$ coordinate in the grid.  The agent can take the actions of moving Up, Down, Left, or Right.  When taking an action the agent has a probability of either the action succeeding or failing and moving perpendicular to the desired direction.  The grid world consists of squares that are either Open, Blocked, Win, or Lose and each has an associated amount of reward.  The grid world ends when either the Win or Lose end states are entered.

#### 2.2.1. Defining the World

In [ ]:
# Set-up the pylab context
%pylab inline
random.seed(0)
# Import libraries that will be used
import copy
import math
from IPython.html.widgets import interact, interactive, fixed
from IPython.html import widgets
from IPython.display import clear_output, display, HTML

In [ ]:
# This is a 3 X 4 Grid World represented by a 3 x 4 array
# Each array location contains a character representing:
#    O = Open location for the agent to traverse
#    B = Location that cannot be traversed by the agent 
#    W = Ending location with the agent winning
#    L = Ending location with the agent losing
world = [['O','O','O','W'],  
         ['O','B','O','L'],
         ['O','O','O','O']]

#### 2.2.2. Setting the Rewards

In [ ]:
# Mapping of current state to reward value
rewards = {}

# Positive reward for entering the win state
rewards['W'] = 1.0 

# Negative reward for entering lost state
rewards['L'] = -1.0 

# No reward for all other states
rewards['O'] = 0.0 
rewards['B'] = 0.0

#### 2.2.3. Initializing the Value Function

In [ ]:
# Value function that defines the utility of being 
# in a particular state in the 3 X 4 grid world, 
# initialized to zero
utility = [[0.0,0.0,0.0,0.0],
           [0.0,0.0,0.0,0.0],
           [0.0,0.0,0.0,0.0]]

# Function to reset all utility values to 0
def reset_util(utility):
    for xx in xrange(len(utility)):
        for yy in xrange(len(utility[xx])):
            utility[xx][yy] = 0.0

#### 2.2.4. Updating the Value Function

Updating the value function is done through the equation:
\begin{align}
\tilde{\mathbf{v}}_{s_i} &= r_{s_i} + \gamma\max_{a\in\mathcal{A}}~\sum_{j=1}^{|\mathcal{S}|}P_{s_i,s_j}^a\left[\tilde{\mathbf{v}}_j\right], ~~\forall s_i\in\mathcal{S}
\end{align}

In [ ]:
# Discount factor (gamma) applied to future rewards
# Smaller value indicates less importance
discount_factor = 1.0

# Updates the utility of a particular grid location given 
#   x,y-coordinate in the world, 
#   current utility values, 
#   world representation, 
#   and the rewards
def update_utility(xx, yy, utility, world):
    
    # Determine the reward for this grid location
    reward = rewards[world[xx][yy]]
        
    # If the world location is blocked, it cannot be visited and has 0 utility
    if(world[xx][yy] == 'B'):
        return 0.0
    
    # If the world location is either the winning or losing, its utility is its reward
    if(world[xx][yy] == 'W' or world[xx][yy] == 'L'):
        return reward
    
    # Else, the utility is the reward of the current location plus the discounted maximum utility from this location 
    return reward + discount_factor * max_utility(xx, yy, utility, world)

And to update the complete value-function, we create a function that loops over all possible states.

In [ ]:
# Updates the utility of each grid location 
# given the current utility values and the world
def update_all_utility(utility, world):
    for xx in range(3):
        for yy in range(4):
            utility[xx][yy] = update_utility(xx, yy, utility, world)

#### 2.2.5. Determining the Action with Max Utility

The code below solves for the part of the equation:
\begin{align}
\max_{a\in\mathcal{A}}~\sum_{j=1}^{|\mathcal{S}|}P_{s_i,s_j}^a\left[\tilde{\mathbf{v}}_j\right], ~~\forall s_i\in\mathcal{S}
\end{align}

In [ ]:
# In the grid world, the probability of success 
# indicates the chance the movement (e.g. Up)
# actually moves up, with a failure resulting
# in a perpendicular direction (e.g. Left or Right)
prob_action_success = 0.8 

# Gets the max utility from the current location in the world
# By observing the utility of each of the action posibilities
# (Up, Down, Left, Right) and selecting the max
def max_utility(xx, yy, utility, world):
    
    # Get the utility from moving up
    # which inludes failures going left and right
    # Any other state contributes 0 to the utility
    # Because they have probability of 0
    up_utility = prob_action_success * action_utility(xx, yy, -1, 0, utility, world) 
    up_utility = up_utility + (1 - prob_action_success)/2 * action_utility(xx, yy, 0, -1, utility, world)
    up_utility = up_utility + (1 - prob_action_success)/2 * action_utility(xx, yy, 0, 1, utility, world)
    
    # Get the utility from moving down
    # which inludes failures going left and right
    # Any other state contributes 0 to the utility
    # Because they have probability of 0
    down_utility = prob_action_success * action_utility(xx, yy, 1, 0, utility, world) 
    down_utility = down_utility + (1 - prob_action_success)/2 * action_utility(xx, yy, 0, -1, utility, world)
    down_utility = down_utility + (1 - prob_action_success)/2 * action_utility(xx, yy, 0, 1, utility, world)
    
    # Get the utility from moving left
    # which inludes failures going up and down
    # Any other state contributes 0 to the utility
    # Because they have probability of 0
    left_utility = prob_action_success * action_utility(xx, yy, 0, -1, utility, world) 
    left_utility = left_utility + (1 - prob_action_success)/2 * action_utility(xx, yy, -1, 0, utility, world)
    left_utility = left_utility + (1 - prob_action_success)/2 * action_utility(xx, yy, 1, 0, utility, world)
    
    # Get the utility from moving right
    # which inludes failures going up and down
    # Any other state contributes 0 to the utility
    # Because they have probability of 0
    right_utility = prob_action_success * action_utility(xx, yy, 0, 1, utility, world) 
    right_utility = right_utility + (1 - prob_action_success)/2 * action_utility(xx, yy, -1, 0, utility, world)
    right_utility = right_utility + (1 - prob_action_success)/2 * action_utility(xx, yy, 1, 0, utility, world)
    
    # Return the max of these utilities
    return max(up_utility, down_utility, right_utility, left_utility)

A helper function is implemented here to address certain edge cases (e.g. moving left from the left most of the world, attempting to move to a blocked grid location).

In [ ]:
# Compute the utility of the new state 
# given a successful action taken from 
# the current state 
def action_utility(xx, yy, xxd, yyd, utility, world):
    
    # Edge cases where the action would move
    # outside the grid world results in staying
    # in the current location
    if(xx + xxd < 0 or xx + xxd > 2 or yy + yyd < 0 or yy + yyd > 3):
        return utility[xx][yy]
    
    # Edge cases where the action would move
    # to a blocked location results in staying
    # at the current location
    if(world[xx + xxd][yy + yyd] == 'B'):
        return utility[xx][yy]
    
    # Normal case, action moves to new state
    # and utility is from the new state
    return utility[xx + xxd][yy + yyd]

#### 2.2.6. Creating Policies

In [ ]:
# Policy mapping defines an action
# to be taken for each state in the
# environment
# In the grid world, these actions are to move:
#   U = Up 
#   D = Down
#   L = Left, 
#   R = Right, 
#   N = Not Applicable (Invalid/End States)
policy = [['U', 'U', 'U', 'N'],
          ['U', 'N', 'U', 'N'],
          ['U', 'U', 'U', 'U']]

We update the policy according to the equation: 
$$
\pi(s_i) =\mathop{\arg\max}_{a\in\mathcal{A}}~\sum_{j=1}^{|\mathcal{S}|}P_{s_i,s_j}^a\left[\hat{\mathbf{v}}_j\right], ~~\forall s_i\in\mathcal{S}.
$$

In [ ]:
# Return the best action to take
# from the current state
# given the current value-function and world
def best_action(xx, yy, utility, world):
    
    # No applicable action if the state puts us in a blocked
    # or ending position
    if(world[xx][yy] == 'B' or world[xx][yy] == 'W' or world[xx][yy] == 'L' ):
        return 'N'
    
    # Get the utility from moving up
    # which inludes failures going left and right
    # Any other state contributes 0 to the utility
    # Because they have probability of 0
    up_utility = prob_action_success * action_utility(xx, yy, -1, 0, utility, world)
    up_utility = up_utility + (1 - prob_action_success)/2 * action_utility(xx, yy, 0, -1, utility, world)
    up_utility = up_utility + (1 - prob_action_success)/2 * action_utility(xx, yy, 0, 1, utility, world)
    
    # Get the utility from moving down
    # which inludes failures going left and right
    # Any other state contributes 0 to the utility
    # Because they have probability of 0
    down_utility = prob_action_success * action_utility(xx, yy, 1, 0, utility, world) 
    down_utility = down_utility + (1 - prob_action_success)/2 * action_utility(xx, yy, 0, -1, utility, world)
    down_utility = down_utility + (1 - prob_action_success)/2 * action_utility(xx, yy, 0, 1, utility, world)

    # Get the utility from moving left
    # which inludes failures going up and down
    # Any other state contributes 0 to the utility
    # Because they have probability of 0
    left_utility = prob_action_success * action_utility(xx, yy, 0, -1, utility, world) 
    left_utility = left_utility + (1 - prob_action_success)/2 * action_utility(xx, yy, -1, 0, utility, world)
    left_utility = left_utility + (1 - prob_action_success)/2 * action_utility(xx, yy, 1, 0, utility, world)
    
    # Get the utility from moving right
    # which inludes failures going up and down
    # Any other state contributes 0 to the utility
    # Because they have probability of 0
    right_utility = prob_action_success * action_utility(xx, yy, 0, 1, utility, world) 
    right_utility = right_utility + (1 - prob_action_success)/2 * action_utility(xx, yy, -1, 0, utility, world)
    right_utility = right_utility + (1 - prob_action_success)/2 * action_utility(xx, yy, 1, 0, utility, world)

    # Initialize the best action
    # to be the first action (Up)
    best_action = 'U'
    
    # Set the maximum utility
    # from any action to up's utility
    max_utility = up_utility
    
    # If moving down has more utility
    if(down_utility > max_utility):
        
        # Set the best action to Down
        max_utility = down_utility
        best_action = 'D'
    
    # If moving left has more utility
    if(left_utility > max_utility):
        
        # Set the best action to Left
        max_utility = left_utility
        best_action = 'L'
    
    #If moving right has the best utility
    if(right_utility > max_utility):
        # Set the best action to Right
        max_utility = right_utility
        best_action = 'R'
        
    # Return the action that maximizes utility
    return best_action

And to sweep across all location, we create a fuction that will loop over the states.

In [ ]:
# Updates the policy with the actions 
# that provide the best utility for each state
def update_policy(policy, world, utility):
    for xx in range(3):
        for yy in range(4):
            policy[xx][yy] = best_action(xx, yy, utility, world)

#### 2.2.7. Running the Algorithm

Bringing together the parts of the algorithm, a function that performs value iteration for a number of iterations, updating the value function and the optimal policy.

In [ ]:
# Performs a number of iterations
# of the value iteration algorithm
def value_iteration(iterations):
    
    # For the number of iterations
    for xx in range(iterations):
        
        # Update the value function
        update_all_utility(utility, world)
        # Update the policy
        update_policy(policy, world, utility)
    
    # Print out the value function
    for xx in range(3):
        print utility[xx]
    print

    # Print out the policy
    for xx in range(3):
        print policy[xx]

Now examine how the algorithm updates the value function and policy by varying the number of iterations.

In [ ]:
# Function that will reset the utility values
# and then perform a number of iterations of
# the value iteration approach
def interactive_iteration(iterations=1):
    reset_util(utility)
    value_iteration(iterations)

# Create an interactive slider for 
# the number of iterations
interact(interactive_iteration, iterations=(0, 100, 1))

#### 2.2.8. Playing with Parameters

#### Reward Values

There are several parameters in this approach that influence the final policy based upon their values.  Among these are the reward values (both the final reward states and the reward for non-final states).  The reward for the final states specifies the ending reward value received for finishing the task in the specified state.  The non-final state rewards represent intermediate rewards during the task and can be applied to influence the policies in different ways.  For example, setting the non-final state reward ('O') to a small negative value causes the policy to attempt to move to the final reward states more quickly.  Experiment with these values below to see how they can significantly change policies.

In [ ]:
# Function that will reset the utility values
# and then perform the value iteration 
# approach with the parameters
def interactive_iteration(winning_reward=1.0,
                          losing_reward=-1.0,
                          intermediate_reward=0.0,
                          iterations=1000):
    # Reward for entering the win state
    rewards['W'] = winning_reward  
    # Reward for entering the lose state
    rewards['L'] = losing_reward
    # Reward for entering an open location
    rewards['O'] = intermediate_reward
    reset_util(utility)
    value_iteration(iterations)
    
# Create interactive sliders for 
# the rward values
interact(interactive_iteration, winning_reward=(0.0, 1.0, 0.05),
                                losing_reward=(-1.0, 0.0, 0.05),
                                intermediate_reward=(-1.0, 1.0, 0.05),
                                iterations=(0,1000,10))

#### Discount Factor

Another parameter that can significantly change the policy is the <i>discount factor</i>.  This parameter controls how distant into the future the policy cares about rewards.  Looking into the future is an important component of such learning because it deals with the <i>credit assignment problem</i>.  The credit assignment problem is the challenge posed by the delayed rewards of a reinforcement learning algorithm that causes difficulty in <i>assigning</i> the <i>credit</i> for the given reward to the correct actions that contributed to receiving it.  A discount factor of one means that there is an infinite horizon on the rewards, that is, the policy cares about all future rewards.  A discount factor of zero means that only immediate rewards matter.  Experiment with this parameter below.

In [ ]:
# Function that will reset the utility values
# and then perform the value iteration 
# approach with the parameters
def interactive_iteration(discount=1.0,
                          winning_reward=1.0,
                          losing_reward=-1.0,
                          intermediate_reward=0.0,
                          iterations=1000):
    # Reward for entering the win state
    rewards['W'] = winning_reward  
    # Reward for entering the lose state
    rewards['L'] = losing_reward
    # Reward for entering an open location
    rewards['O'] = intermediate_reward
    global discount_factor
    discount_factor = discount
    reset_util(utility)
    value_iteration(iterations)
    
# Create interactive sliders for 
# the rward values
interact(interactive_iteration, discount=(0.0, 1.0, 0.05),
                                winning_reward=(0.0, 1.0, 0.05),
                                losing_reward=(-1.0, 0.0, 0.05),
                                intermediate_reward=(-1.0, 1.0, 0.05),
                                iterations=(0,1000,10))

#### Transition Probabilities

Finally, the transition probability can significantly change the policies.  In this case, we have a probability of the chosen action succeeding or moving in a perpendicular direction.  By decreasing the probability of success, we increase the uncertainty in the world, and by increasing the probability, the world's uncertainty decreases.  Observe how varying the level of uncertainty changes the policy below.

In [ ]:
# Function that will reset the utility values
# and then perform the value iteration 
# approach with the parameters
def interactive_iteration(success=1.0,
                          discount=0.8,
                          winning_reward=1.0,
                          losing_reward=-1.0,
                          intermediate_reward=0.0,
                          iterations=1000):
    # Reward for entering the win state
    rewards['W'] = winning_reward  
    # Reward for entering the lose state
    rewards['L'] = losing_reward
    # Reward for entering an open location
    rewards['O'] = intermediate_reward
    global discount_factor
    discount_factor = discount
    global prob_action_success
    prob_action_success = success
    reset_util(utility)
    value_iteration(iterations)
    
# Create interactive sliders for 
# the rward values
interact(interactive_iteration, success=(0.0, 1.0, 0.05),
                                discount=(0.0, 1.0, 0.05),
                                winning_reward=(0.0, 1.0, 0.05),
                                losing_reward=(-1.0, 0.0, 0.05),
                                intermediate_reward=(-1.0, 1.0, 0.05),
                                iterations=(0,1000,10))

### 2.3. Temporal Difference Learning

Often, MDP models are not known either because the underlying environment is poorly characterized or because the dynamics of the environment are too complex to be captured analytically. To address such learning scenarios with an unknown environment, the agent must dynamically interact with the environment using a policy, $\pi$ in a sequence of discrete time steps, $t\in\mathbb{N}$. At each time step $t$, the learning agent observes the environment's state, $s_{(t)}\in\mathcal{S}$ and selects an action $a_{(t)}\in\mathcal{A}$ based upon $\pi(s_{t})$. The next time step, the learning agent receives a numerical reward $r_{(t+1)}$ depending on its new state $s_{(t+1)}$.

The goal of RL in this domain is to solve the planning problem of an MDP solely through interactions with the environment and without knowledge of the model that characterizes the MDP. Compared to the planning problem of a fully-known MDP, an RL agent must overcome the following challenges:

- The learning agent does not have access to all states at once. Thus, samples are gathered as part of a trajectory.
- The learning agent cannot directly access the model parameters, i.e., $\mathbf{P}$ and $\mathbf{r}$.
- The per-time-step complexity is limited by the time between interactions with the environment.

Due to their reliance on samples, such methods are sensitive to the exploration methods used. The exploration method determines how the learning agent seeks out new samples from the environment while still attempting to maximize reward.

These methods that learned from a sequence of events are collectively known as Temporal-Difference (TD) Learning, since they rely on reward differences in time to learn policies. At time $t$, TD methods update the value function estimates using the observed reward $r_{(t)}$ at time $t$, and the estimate of $V(s_{(t+1)})$, where $s_{(t+1)}$ denotes the state observed by the system at $t+1$. The update followed by the simplest TD method, known as TD(0), is

$$
V(s_{(t)})\leftarrow V(s_{(t)})+\alpha\left[r_{(t)}+\gamma V(s_{(t+1)})-V(s_{(t)})\right].
$$

Note that the TD method bases its updates in part on an existing estimate of the state value function $V$.  Additionally, observe the parameter $\alpha$ that represents the <i>learning rate</i>, that is, how much of an impact newly observed information has versus prior estimates. TD methods do not require a model of the MDP, are naturally implemented in an on-line and fully incremental fashion, and can learn from each transition regardless of what subsequent actions are taken.

Let us now consider how TD methods can be used in RL. The main idea is to proceed according to the notion of a generalized policy iteration (GPI). In GPI one maintains both an approximate policy and an approximate state value function. The state value function is repeatedly altered to more closely approximate the state value function for the current policy, and the policy is repeatedly improved with respect to the current $V$. If a model is not available, then it is particularly useful to estimate a state-action value function $Q$ rather than $V$. With a model, $V$ alone is sufficient to determine a policy by simply looking ahead one step and choosing whichever action leads to the best combination of reward and next state. Without a model, however, $V$ alone is not sufficient. One must explicitly estimate the value of each action in order for the values to be useful in suggesting a policy. 

The policy evaluation problem for a state-action value function is to estimate $Q^{\pi}(s,a)$, that is, the expected return when starting in state $s$, taking action $a$, and thereafter following policy $\pi$. Policy improvement is done by using a greedy policy with respect to the current value function. In this case, one has a state-action value function $Q$, and therefore no model is needed to construct the greedy policy. For any $Q$, the corresponding greedy policy is the one that, for each $s\in\mathcal{S}$, deterministically chooses an action with maximal $Q$-value as

$$
\pi(s)=\mathop{\arg\max}_{a\in\mathcal{A}}Q(s,a).
$$

Another policy option that strikes a balance between exploration and exploitation is to use an $\varepsilon$-greedy policy, which chooses random actions every time with a small probability $\varepsilon\in[0,1]$, and acts greedily otherwise as

$$
\pi^{\varepsilon}(s)=\left\{\begin{array}{ll}
\mathop{\arg\max}_{a\in\mathcal{A}}Q(s,a)&\textrm{with probability 1-$\varepsilon$}\\
\textrm{Choose $a\in\mathcal{A}$ uniformly at random}&\textrm{with probability $\varepsilon$}
\end{array}\right..
$$

Next, we will explore two important instatiations of TD(0) learning.

### 2.3a. Q-learning: Off-policy TD learning

In off-policy learning, the policy used to generate behavior, called the behavior policy, may in fact be unrelated to the policy that is evaluated and improved, called the estimation policy. An advantage of this separation is that the estimation policy may be simple to evaluate (e.g., a greedy policy), while the behavior policy can continue to sample all possible actions.

$Q$-learning uses an $\varepsilon$-greedy policy as its behavior policy, which guarantees that all actions are eventually sampled, and a greedy policy for estimating $Q^*$. $Q$-learning updates the $Q$-values as
$$
Q(s_{(t)},a_{(t)})\leftarrow Q(s_{(t)},a_{(t)})+\alpha\left[r_{(t)}+\gamma \max_{a\in\mathcal{A}}Q(s_{(t+1)},a)-Q(s_{(t)},a_{(t)})\right].
$$

To demonstrate, we will use the prior grid world domain.  Below are the necessary additions to perform $Q$-learning are implemented.

#### 2.3a.1 Interacting with the World

Because such approached learn by interacting with the world, first we create the means of tracking current state and performing actions to transition from one state to the next.

In [ ]:
# Starting state
# Represents the row and column
# the agent starts at in the grid world
start_x = 2
start_y = 0

# Current state
# Represents the row and column
# the agent is currently at in the grid world
# Initialized to the start state
cur_x = start_x
cur_y = start_y

In [ ]:
# Function that performs an action
# in the grid world and then returns
# the new state
# Available actions are (U)p, (D)own, (L)eft, and (R)ight
def perform_action(xx, yy, action):
    
    # If in any of these states, cannot move from them (Blocked, Win, Lost)
    if(world[xx][yy] == 'B' or world[xx][yy] == 'W' or world[xx][yy] == 'L'):
        # Reamin at current location
        return xx,yy
    
    # Determine if move does not succeed with given probability
    if(random.uniform(0, 1) > prob_action_success):
        
        # If original action is up or down, 
        # change the action to left on failure
        if(action == 'U' or action == 'D'):
            action = 'L'
        
        # Else original action is left or right
        # change the action to up
        else:
            action = 'U'
        
        # Flip the direction of the failure
        # with equal probability (e.g. either
        # fail left/right, or up/down)
        if(random.uniform(0, 1) < 0.5):
            # Reverse directions
            if(action == 'U'):
                action = 'D'
            else:
                action = 'R'
    
    # Set the new state values
    # to the current state values
    new_x = xx
    new_y = yy
    
    # Perform the movement by
    # adding/substracting from
    # appropriate state variables
    if(action == 'U'):
        new_x = new_x - 1
    elif(action == 'D'):
        new_x = new_x + 1
    elif(action == 'L'):
        new_y = new_y - 1
    elif(action == 'R'):
        new_y = new_y + 1
    
    # If the new state is out of bounds, 
    # or trying to move to a blocked location
    # return the original state
    if(new_y < 0 or new_y > 3 or new_x < 0 or new_x > 2):
        return xx,yy
    if(world[new_x][new_y] == 'B'):
        return xx,yy
    
    # Return the new state
    return new_x,new_y

#### 2.3a.2 The Q-Table

The foundation of Q-Learning is the Q-table, which defines the reward for a given state and action: $Q(s,a)$.

In [ ]:
# Q-table that contains the
# approximation of the state-action 
# value function
q_values = {}

# Each action maps to a matrix of values
# that represent the value of taking
# that action in a particular state
# Thus, q_values['U'][1][3] returns the
# approximate reward for moving Up
# when in the 1st row, 3rd column
q_values['U'] = [[0, 0, 0, 0],
                [0, 0, 0, 0],
                [0, 0, 0, 0]]
q_values['D'] = [[0, 0, 0, 0],
                [0, 0, 0, 0],
                [0, 0, 0, 0]]
q_values['L'] = [[0, 0, 0, 0],
                [0, 0, 0, 0],
                [0, 0, 0, 0]]
q_values['R'] = [[0, 0, 0, 0],
                [0, 0, 0, 0],
                [0, 0, 0, 0]]

# Function to initialize the q-table
# entries to a particular value
def init_qtable(init_value):
    for xx in range(3):
        for yy in range(4):
            q_values['U'][xx][yy] = init_value
            q_values['D'][xx][yy] = init_value
            q_values['L'][xx][yy] = init_value
            q_values['R'][xx][yy] = init_value

#### 2.3a.3 Choosing Actions

To interact with the environment, Q-Learning selects an action based upon the values in the Q-Table and a parameter for randomness, epsilon ($\epsilon$).  This approach is known as the $\epsilon$-greedy approach.

$$
\pi^{\varepsilon}(s)=\left\{\begin{array}{ll}
\mathop{\arg\max}_{a\in\mathcal{A}}Q(s,a)&\textrm{with probability 1-$\varepsilon$}\\
\textrm{Choose $a\in\mathcal{A}$ uniformly at random}&\textrm{with probability $\varepsilon$}
\end{array}\right..
$$


In [ ]:
# Rate of selecting action at random, 
# rather than greedy selection
epsilon = 0.1

# Chooses an action through the epsilon greedy approach
def choose_action(xx, yy, q_table):
    
    # If we choose at random
    if(random.uniform(0,1) < epsilon):
        # Select Up, Down, Left, or Right
        # in a uniform random manner
        selection = random.uniform(0,1)
        if(selection < 0.25):
           return 'U'
        elif(selection < 0.5):
           return 'R'
        elif(selection < 0.75):
           return 'D'
        else:
            return 'L'
        
    # Else select the best action possible (greedy)
    return best_action(xx, yy, q_table)

And to select the best action from the Q-table ($\mathop{\arg\max}_{a\in\mathcal{A}}Q(s,a)$) is implemented below.

In [ ]:
# Selects the best action 
# at the given state through the given q-table
def best_action(xx, yy, q_table):
    best = 'U'
    if(q_table[best][xx][yy] < q_table['R'][xx][yy]):
        best = 'R'
    if(q_table[best][xx][yy] < q_table['D'][xx][yy]):
        best = 'D'
    if(q_table[best][xx][yy] < q_table['L'][xx][yy]):
        best = 'L'
    return best

#### 2.3a.4 Updating the Q-values

Q-values are updated according to the equation:
$$
Q(s_{(t)},a_{(t)})\leftarrow Q(s_{(t)},a_{(t)})+\alpha\left[r_{(t)}+\gamma \max_{a\in\mathcal{A}}Q(s_{(t+1)},a)-Q(s_{(t)},a_{(t)})\right].
$$


In [ ]:
# Learning rate of (alpha) Q-Learning
learning_rate = 0.9
# Discount factor (gamma) applied to future rewards
discount_factor = 0.9

# Updates the Q-table given the current state, action taken,
# and the next state according the the learning rate
# and discoutn factor
def update_q_table(q_table, xx1, yy1, action, xx2, yy2):    
    
    # If it is a final state
    if(world[xx1][yy1] != 'O'):
        # The q-value is simply the reward
        q_table[action][xx1][yy1] = rewards[world[xx1][yy1]]
        return
    
    # Find the optimal action of the next state
    best = best_action(xx2, yy2, q_table)
    
    # Find the update value for the Q value
    update = learning_rate * (rewards[world[xx1][yy1]] + discount_factor * q_table[best][xx2][yy2] - q_table[action][xx1][yy1])
    
    # Update the Q value for the action at the original state
    q_table[action][xx1][yy1] = q_table[action][xx1][yy1] + update

#### 2.3a.5 Deriving Policy from Q-values

The policy is derived from the Q-values through:
$$
\pi(s)=\mathop{\arg\max}_{a\in\mathcal{A}}Q(s,a).
$$

In [ ]:
# Updates the optimal policy with the optimal actions given the Q-values
def update_policy(policy, q_values):
    for xx in range(3):
        for yy in range(4):
            policy[xx][yy] = 'N'
            if(world[xx][yy] == 'O'):
                policy[xx][yy] = best_action(xx,yy,q_values)

#### 2.3a.5 Assembling the Q-Learning Algorithm

In [ ]:
# This function performs q-learning
# for a given number of iterations
def q_learn(iterations):
  
    cur_x = start_x
    cur_y = start_y
    
    # For the number of iterations
    for ii in range(iterations): 
        
        # Choose an action (epsilon greedy)
        action = choose_action(cur_x, cur_y, q_values)
        
        # Perform the action and move to the new state
        new_x, new_y = perform_action(cur_x, cur_y, action)
        
        # Update the Q-table
        update_q_table(q_values, cur_x, cur_y, action, new_x, new_y)
        
        # If in the end state, reset to the starting state
        if(cur_x == new_x and cur_y == new_y and world[cur_x][cur_y] != 'O'):
            new_x = start_x
            new_y = start_y
            
            
        # Set the current state to the new state
        cur_x,cur_y = new_x,new_y

#### 2.3a.6 Playing with Parameters

There are two new parameters introduced into the Q-Learning paradigm: The learning rate, $\alpha$, and the randomness parameter, $\epsilon$. Change epsilon to accelerate the rate of exploration and the learning rate to change how quickly the Q values update with new information.  Other paramters from the value iteration approach remain and can have significant effect on Q-Learning.  Explore these interactions below.

In [ ]:
# Function that will reset the utility values
# and then perform the value iteration 
# approach with the parameters
def interactive_iteration(learning = 0.25,
                          randomness = 0.1,
                          success=0.85,
                          discount=0.9,
                          winning_reward=1.0,
                          losing_reward=-1.0,
                          intermediate_reward=0.0,
                          iterations=1000):
    global learning_rate
    learning_rate = learning
    global epsilon
    epsilon = randomness
    # Reward for entering the win state
    rewards['W'] = winning_reward  
    # Reward for entering the lose state
    rewards['L'] = losing_reward
    # Reward for entering an open location
    rewards['O'] = intermediate_reward
    global discount_factor
    discount_factor = discount
    global prob_action_success
    prob_action_success = success
    
    # Initializes the Q-table to be all 1's
    init_qtable(1.0)

    # Performs Q-Learning for iterations
    q_learn(iterations)

    # Updates the policy based on the Q-table
    update_policy(policy, q_values)

    # Print the Q-values for each action
    print 'Up'
    for xx in range(3):
        print q_values['U'][xx]
    print
    print 'Down'
    for xx in range(3):
        print q_values['D'][xx]
    print
    print 'Left'
    for xx in range(3):
        print q_values['L'][xx]
    print
    print 'Right'
    for xx in range(3):
        print q_values['R'][xx]
    print

    # Print out the policy
    print 'Policy'
    for xx in range(3):
        print policy[xx]   
    
# Create interactive sliders for 
# the rward values
interact(interactive_iteration, learning=(0.0, 1.0, 0.05),
                                randomness=(0.0, 1.0, 0.05),
                                success=(0.0, 1.0, 0.05),
                                discount=(0.0, 1.0, 0.05),
                                winning_reward=(0.0, 1.0, 0.05),
                                losing_reward=(-1.0, 0.0, 0.05),
                                intermediate_reward=(-1.0, 1.0, 0.05),
                                iterations=(0,10000,10))

### 2.3b. SARSA: On-policy TD learning

Alternatively, on-policy methods attempt to evaluate or improve the policy that is used to make decisions. In on-policy methods the policy is generally soft, meaning that there is a non-zero probability to select every feasible action while in state $s$. There are many possible variations on on-policy methods. Here, we focus on $\varepsilon$-greedy policies, meaning that most of the time they choose an action that has maximal estimated action value, but with probability $\varepsilon/|\mathcal{A}|$, with $\varepsilon>0$, they instead choose an action at random. 

In particular, we examine an algorithm called State, Action, Reward, State, Action (SARSA). An agent using SARSA interacts with the environment and updates the policy based on actions taken. The $Q$-value for a state-action pair is sequentially updated by an error quantity, adjusted by the learning rate $\alpha>0$ which determines to what extent newly acquired information overrides old information. Note that SARSA learns the $Q$-values associated with taking the policy it follows. SARSA uses an $\varepsilon$-greedy policy and updates the $Q$-values as

$$
Q(s_{(t)},a_{(t)})\leftarrow Q(s_{(t)},a_{(t)})+\alpha\left[r_{(t)}+\gamma Q(s_{(t+1)},a_{(t+1)})-Q(s_{(t)},a_{(t)})\right].
$$

where $a_{(t)}$ denotes the action taken at time $t$. This rule uses every element of the quintuple of events, $(s_{(t)},a_{(t)},r_{(t)},s_{(t+1)},a_{(t+1)})$, that make up a transition from one state-action pair to the next. This quintuple gives rise to the name SARSA for the algorithm.  Below, we define the methods required for SARSA learning.

In [ ]:
# Updates the Q-table given the current state, action taken, and the next state
def update_q_table_sarsa(q_table, xx1, yy1, action1, xx2, yy2, action2):    
    
    # If in a final state
    if(world[xx1][yy1] != 'O'):
        # Q-value is the reward
        q_table[action1][xx1][yy1] = rewards[world[xx1][yy1]]
        return
    
    # Find the update value for the Q value
    update = learning_rate * (rewards[world[xx1][yy1]] + discount_factor * q_table[action2][xx2][yy2] - q_table[action1][xx1][yy1])
    
    # Update the Q value for the action at the original state
    q_table[action1][xx1][yy1] = q_table[action1][xx1][yy1] + update

As you can see, it is a minor change from $Q$-learning to SARSA.  Instead of assuming the next action taken is the optimal, The Q-values are updated according the next action that was chosen.  All the same parameters from before apply, investigate below.

In [ ]:
# This function performs SARSA
# for a given number of iterations
def sarsa(iterations):
  
    cur_x = start_x
    cur_y = start_y
    prev_x = start_x
    prev_y = start_y
    prev_action = -1
        
    # For the number of iterations
    for ii in range(iterations): 
        
        # Choose an action (epsilon greedy)
        next_action = choose_action(cur_x, cur_y, q_values)
        
        if(prev_action != -1):
            # Update the Q-table
            update_q_table_sarsa(q_values, prev_x, prev_y, prev_action, cur_x, cur_y, next_action)
      
        
        # Perform the action and move to the new state
        new_x, new_y = perform_action(cur_x, cur_y, next_action)
        
        # If in the end state, reset to the starting state
        if(cur_x == new_x and cur_y == new_y and world[cur_x][cur_y] != 'O'):
            new_x = start_x
            new_y = start_y
            prev_action = -1
        
        # Set the current state to the new state
        prev_action = next_action
        prev_x,prev_y = cur_x,cur_y
        cur_x,cur_y = new_x,new_y

In [ ]:
# Function that will reset the utility values
# and then perform the value iteration 
# approach with the parameters
def interactive_iteration(learning = 0.2,
                          randomness = 0.1,
                          success=0.85,
                          discount=0.8,
                          winning_reward=1.0,
                          losing_reward=-1.0,
                          intermediate_reward=0.0,
                          iterations=1000):
    global learning_rate
    learning_rate = learning
    global epsilon
    epsilon = randomness
    # Reward for entering the win state
    rewards['W'] = winning_reward  
    # Reward for entering the lose state
    rewards['L'] = losing_reward
    # Reward for entering an open location
    rewards['O'] = intermediate_reward
    global discount_factor
    discount_factor = discount
    global prob_action_success
    prob_action_success = success
    
    # Initializes the Q-table to be all 1's
    init_qtable(1.0)

    # Performs Q-Learning for iterations
    sarsa(iterations)

    # Updates the policy based on the Q-table
    update_policy(policy, q_values)

    # Print the Q-values for each action
    print 'Up'
    for xx in range(3):
        print q_values['U'][xx]
    print
    print 'Down'
    for xx in range(3):
        print q_values['D'][xx]
    print
    print 'Left'
    for xx in range(3):
        print q_values['L'][xx]
    print
    print 'Right'
    for xx in range(3):
        print q_values['R'][xx]
    print

    # Print out the policy
    print 'Policy'
    for xx in range(3):
        print policy[xx]   
    
# Create interactive sliders for 
# the rward values
interact(interactive_iteration, learning=(0.0, 1.0, 0.05),
                                randomness=(0.0, 1.0, 0.05),
                                success=(0.0, 1.0, 0.05),
                                discount=(0.0, 1.0, 0.05),
                                winning_reward=(0.0, 1.0, 0.05),
                                losing_reward=(-1.0, 0.0, 0.05),
                                intermediate_reward=(-1.0, 1.0, 0.05),
                                iterations=(0,10000,10))

### 2.4. TD-Learning Conclusion

This ends our introduction to temporal difference learning.  We completed an overview of a basics of Markov Decision Process, vale-function learning and demonstrated the key components of Temporal Difference Learning algorithms ($Q$-Learning and SARSA).  As a reminder, the key parts to such approaches include:

<ul>
<li><b>Environment</b>:        World with which a learning agent interacts</li>
       <li><b>State ($s$)</b>:        Observations that define the environment for the agent</li>
       <li><b>State-Space ($S$)</b>:  All possible values of the state</li>
       <li><b>Action ($a$)</b>:       Decision by agent that interacts with the environment</li>
       <li><b>Action-Space ($A$)</b>: All possible actions the agent may make</li>
       <li><b>State Transition:</b>   Moving from one state to another state given an action</li>
       <li><b>Policy ($\pi$)</b>:     Mapping of states to actions ($\pi(s)=a$)</li>
       <li><b>Reward ($r$)</b>:       Benefit received from a particular state</li>
</ul>

There is a vast amount of literature out there if you wish to learn more, from basic text books like <i>Reinforcement Learning: an introduction</i> by Richard S. Sutton and Andrew G. Barto, to cutting edge international conferences, such as the International Conference on Machine Learning (ICML) and the Autonomous Agents and Multi-Agent Systems conference (AAMAS).  

This has been a small taste of the rich theory and applications that exists in TD-Learning research.  Additions to address topics such as continuous and large state spaces include tile-coding and using fuction approximators to estimate rewards for continuous states.  One of the most important areas of current research is to eliminate or loosen the foundational assumptions that allow such methods to work, such as the state representation requiring the Markov property (see POMDPs).

## 3. Evolutionary Computation

In many ways, evolution can be considered the ultimate reinforcement learning algorithm.  All it cares about is the propagation of genes, that is, how well does a living being replicate and allow their genes to persist. Every living being (agent) takes actions in response to stimulus from their environment (state); however, we do not necessarily know how much any particular action contributes to the propagation of their genes (reward).  

Evolutionary approaches have several advantages over other reinforcement learning approaches ([Schmidhuber, 2000](ftp://ftp.idsia.ch/pub/juergen/seal2000.pdf)). In particular, unlike value-function approaches, evolutionary approaches do not rely on Markov assumptions, such as full observability of the world.  For an idea of the breadth and depth of research considered part of evolutionary computation, please check out the Genetic and Evolutionary Computation Conferences (GECCOs), hosted by [ACM's Special Interest Group in Evolution (SIGEVO)](http://www.sigevo.org/).

### 3.1. Overview and Key Terms

Evolutionary computation mainly began in the 1960's-70's in three separate strains of research.  The first, evolutionary programming focused on creating a system that would automate the process of programming software.  This automation was performed through an evolutionary process applied to finite state machines.  Second, evolutionary strategies focused on using evolution to optimize real-valued parameters for problem domains.  Finally, genetic algorithms examined directly evolving the binary representation of a solution to a problem in order to solve it.  Starting in the 1990's, these fields recognized they were highly related and began to unify. For a more complete history, see [Evolutionary Computation: a unified approach](http://mitpress.mit.edu/books/evolutionary-computation) by Kenneth De Jong.


Despite the varied background of the beginnings of evolutionary computation, they all shared several characteristics:
<ul>
<li><b>Individuals (or, Genomes):</b> Candidate solution instances to the problem domain</li>
<li><b>Populations:</b> Sets of individuals</li>
<li><b>Fitness:</b> A measure of an individual's performance in the problem domain</li>
<li><b>Reproduction:</b> Process by which existing individuals (<b>parents</b>) produce new individuals (<b>offspring, or children</b>)</li>
<li><b>Selection:</b> A process by which either (1) parents are chosen to create offspring, or, (2) individuals are chosen to die and be replaced </li>
</ul>

And all follow a similar algorithmic outline:

<ul>
<li><b>BEGIN Evolution:</b>
    <ul>
    <li>Initialize population (Create set of random genomes and evaluate)</li>
    <li><b>DO</b>:
    <ul>
        <li>Select parents from population</li>
        <li>Reproduce parents to create offspring</li>
        <li>Evaluate offspring</li>
        <li>Select individuals to make next population from current population + offspring</li> 
    </ul>
    </li>
    <li><b>WHILE</b>: Stopping criteria not met</li>
    </ul>
<li><b>DONE</b></li>
</ul>

From this simple loop, a multitude of approaches can be implemented by varying the details of each step.  For evaluation, not only can the domain change, but alternatives such as co-evolution (individuals compete against each other) or real-time evolution (the population is continuously evaluated, with offspring added to the ongoing evaluation).  Reproduction can be applied through <i>Mutation</i>, where an existing genome is cloned and then has random changes made to it, or, <i>Recombination</i>, where two or more genomes have their genes jumbled together.  Mutation is also called <i>Asexual Reproduction</i> and Recombination can be called <i>Crossover</i> or <i>Sexual Reproduction</i>.  Finally, selection approaches can vary from uniform random selection (neutral) to truncation selection (high pressure).  

All of these topics will be explored in the following exercises.

### 3.2 Constructing an Evolutionary Algorithm

#### 3.2.1. The Genome

In these exercises, we will construct a basic evolutionary algorithm by exploring function optimization through the evolution of real-valued parameters.  To begin, we need to define the form our solution takes:

In [ ]:
# Genome class encapsulates information about a genome
# including fitness and the genetic representation 
# of the solution plus useful utility functions
class Genome: 
    # Initializer for new genome instance
    def __init__(self, fitness, genes):
        # Set the instance's fitness to the given fitness
        self.fitness = fitness
        # Set the instance's genese to the given genes
        self.genes = genes 
    
    # Creates a deep clone of the genome
    def clone(self): 
        # Uses copy's deepcopy function to create a deep copy of this Genome instance
        return copy.deepcopy(self) 

In this case, the genome consists of a fitness value and a set of genes.  For the next few experiments, we will define the set of genes as a single value $x\in\mathbb{R}$.  Below, we define a function to generate a new, random genome with $x\in[-1,1]$.

In [ ]:
# Creates and returns a new random genome
def new_genome():   
    # A random genome for this domain is a real valued number (-1, 1)
    return Genome(-10000, random.uniform(-1, 1)) 

#### 3.2.2. Evaluating Performance

Closely linked to the genes that represent the solution is how performance is evaluated.  This means of evaluation can also be called the domain.  In this example, we are trying to optimize the function $f(x)= 1.0 - x^2$, were $x$ is the single real-valued gene of the genome. Thus fitness can simply be described as the result of the function.

In [ ]:
# Evaluator that returns fitness of the genome
def evaluate(genome): 
    # This fitness is equal to 1 - x^2
    genome.fitness = 1.0 - genome.genes * genome.genes 

#### 3.2.3. (un)Natural Selection

After specifying what a genome is and how it is evaluated, the next step becomes selecting among genomes based upon their fitness score.  In evolutionary algorithms this can be done at two different points (1) Selecting parents to produce offspring, and, (2) Selecting individuals that survive to the next generation.

#### Selecting Parents

In this case, we give each genome in the population an equal probability of being selected as a parent.

In [ ]:
# Selects the given number of parents from the given population
def select_parents(population, number): 
    
    # Initialize an empty list of parents
    parents = []  
    
    # Get the number of individuals in the given population
    populationSize = len(population)
    
    # For the number of parents
    for x in range(number): 
        
        # Select a random genome from the population
        # with uniform probability to be a parent
        parents.append(population[random.randint(populationSize)]) 
    
    # Return the parents list
    return parents 

#### Selecting: The Next Generation

The second part of selection is determining which genomes survive from one generation to the next.  In this instance, select a set number of genomes (the population size) from the combined pool of the existing population and newly created offspring through a process called <i>tournament selection</i>.  Tournament selection works by randomly selecting two genomes with uniform probability from the pool and selecting the genome with greater fitness to take a place in the next generation.

In [ ]:
# Select a new population of individuals from the current and offspring
def select_population(currentPopulation, offspring): 
    
    # Get the size of the current population
    popSize = len(currentPopulation)
    
    # Initialize an empty list for the new population
    newPopulation = [] 
    
    # Combine the current and offspring populations
    currentPopulation.extend(offspring) 
    
    # Get the combined size of current and offspring populations
    combinedSize = len(currentPopulation)
    
    # Until the population for the next generation
    # is the size of the current population
    # keep selecting genomes
    while(len(newPopulation) < popSize): 
        
        # Select an index for the first candidate genome
        candidateOneIdx = random.randint(combinedSize)  
        # Select an index for the second candidate genome
        candidateTwoIdx = random.randint(combinedSize)  
        
        # If the same index is selected
        if(candidateOneIdx == candidateTwoIdx): 
            # Increment second index by one and make sure it is within bounds
            candidateTwoIdx = (candidateTwoIdx + 1)%combinedSize
            
        # Set the selected genome to the first candidate
        selected = currentPopulation[candidateOneIdx]  
        
        # If the second candidate's fitness is better
        # set it to be selected
        if(currentPopulation[candidateTwoIdx].fitness > selected.fitness): 
            selected = currentPopulation[candidateTwoIdx]
            
        # Add the selected genome to the new population
        # and remove it from the pool of candidates
        newPopulation.append(selected) 
        currentPopulation.remove(selected)
        
        # Decrement the pool count
        combinedSize -= 1
        
    # Return the new population
    return newPopulation 

#### 3.2.4. Creating Offspring

Another aspect closely linked to the problem domain and genetic representation of the solution is how to create new genomes from the existing population.  Broadly speaking, there are two main approaches to generating new solutions, <i>mutation</i> (applied here), and, <i>crossover</i> (discussed later).  Mutation works by making a clone of a parent and then perturbing their genes through some process, often randomly.  Below are two functions: One that clones and mutates each of the parents, and, One that defines how the genes will be mutated.  In this case, the mutation will be a random perturbation of the real-valued gene in the range $[-0.01,0.01]$. 

In [ ]:
# Takes in a set of parents and produces 
# an equivalent number of offspring
def reproduce(parents):
    
    # Initialize an empty list of offspring
    offspring = [] 
    
    # For each parent, create a clone
    # and mutate the clone
    for parent in parents: 
        
        # Clone the parent to produce a child
        child = parent.clone() 
        
        # Set its fitness to a very low value
        child.fitness = -10000 
        
        # Mutate the clone child
        mutate_genome(child) 
        
        # Add the child to the offspring list
        offspring.append(child) 
        
    # Return the list of offspring
    return offspring 

In [ ]:
# Mutation takes a genome and alters 
# its genes to create a new solution instance
def mutate_genome(genome): 
    
    # This mutatation adds a value between 
    # -0.01 and 0.01 to the genome's genes
    genome.genes += random.uniform(-0.01, 0.01) 

#### 3.2.5. The Loop of Evolution

Finally, we are ready to construct the evolutionary loop.  In this implementation, we are going to define a function that takes sizes for population and number of offspring and a maximum number of iterations to perform. Note: Each iteration of evolution is called a <i>generation</i>.  In this example, evolution will stop after the set number of generations.  During evolution, the loop will keep track of the best genome found, the average and best fitness at each generation, and genes of the best genome at each generation.  These values will be plotted once evolution has been completed. The function will then return the genome that achieved the best fitness.

In [ ]:
# Function will run a set number of iterations of evolution
# with a given population size and number of offspring
# to create each generation
def evolution(populationSize, offspringSize, maxGenerations):
    
    # Current generation being executed
    currentGeneration = 0 
    
    # Array to hold generational average fitness values
    avgFitnessByGeneration = [] 
    # Array to hold generational champion fitness values
    bestFitnessByGeneration = [] 
    # Array to hold generational champion values
    championValuesByGeneration = [] 
    
    # Initialize an empty population
    population = []  
    # Set the best genome to a null value
    bestGenome = None 
    
    # Initialize variable for calculating population average fitness to 0
    avg = 0.0  
    
    # Create and evaluate an initial population
    # of genomes
    for x in range(populationSize):
        
        # Create new random genome and add it to the population
        population.append(new_genome())
        # Evaluate that new genome
        evaluate(population[x]) 
        # Add its fitness score to the average
        avg += population[x].fitness 
        
        # If the best genome hasn't been set, or, this
        # genome has better fitness
        if(bestGenome is None or population[x].fitness > bestGenome.fitness):  
            
            # Set the bestGenome to this genome
            bestGenome = population[x] 
    
    # Add the metrics to their various arrays for tracking
    avgFitnessByGeneration.append(avg/populationSize)
    bestFitnessByGeneration.append(bestGenome.fitness)
    championValuesByGeneration.append(bestGenome.genes)
    
    # Until the maxmium number of generations 
    # is performed
    while(currentGeneration < maxGenerations): 
        
        # Select a number of parents equal 
        # to the number of offspring to be created
        parents = select_parents(population, offspringSize)
        
        # Create a number of offspring by 
        # reproducing the selected parents
        offspring = reproduce(parents) 
        
        # Evaluate each of the offspring
        for child in offspring:
            
            evaluate(child) 
            
            # Update the best genome is applicable
            if(child.fitness > bestGenome.fitness): 
                bestGenome = child
        
        # Select a new population from the 
        # current population and offspring
        population = select_population(population, offspring) 
        
        # Initialize average to 0
        avg = 0.0 
        
        # Sum the fitness of the population
        for genome in population: 
            avg += genome.fitness 
   
        # Add the metrics to the various arrays
        avgFitnessByGeneration.append(avg/populationSize)
        bestFitnessByGeneration.append(bestGenome.fitness)
        championValuesByGeneration.append(bestGenome.genes)
        
        # Increment current generation by one
        currentGeneration += 1 
    
    # Evolution Loop Done
    
    # Plot the best fitness, average fitness, and the champion gene values by generation
    labels = range(len(bestFitnessByGeneration))
    subplot(2,1,1)
    plot(labels, bestFitnessByGeneration, labels, avgFitnessByGeneration)
    ylabel("Fitness")
    legend(["Best", "Avg."], loc='lower center', bbox_to_anchor=(0.5, 1.05),
          ncol=3, fancybox=True, shadow=True)
    subplot(2,1,2)
    plot(labels, championValuesByGeneration)
    ylabel("Gene Values")
    
    # Return the best genome
    return bestGenome 

#### 3.2.6. Test Run

Now that we have implemented our evolutionary algorithm, let us test it with a population size of one, that produces one offspring, and runs for 1000 generations.  At the end, the genes and the fitness of the best individual will be output, along with a graph of performance over the generations. Because we are optimizing for $f(x) = 1 - x^2$, the optimal value will  be $x = 0$, producing a fitness of $1.0$. Note: Because the population and offspring are both size one, the average fitness is equal to the best fitness.

In [ ]:
# Run evolution with a population of size 1
# and creting 1 offspring per generation
# for 1000 generations
winner = evolution(1, 1, 1000)

# Print out the best genome's genese and fitness
print("Champion -- Genes = " + str(winner.genes) + " Fitness = " + str(winner.fitness))


### 3.3. Exercises

#### 3.3.1. The Evaluation (Fitness) Function

Now, that the algorithm is complete, we will explore different aspects of the evolutionary approach.  One of the strengths of the evolutionary approach is its minimial assumptions about the domain and the simple need to receive feedback from the evaluation function. This advantage means it is possible to alter the evaluation function without changing any other part of the algorithm. Try out your own fitness functions by changing the evaluate function below:

In [ ]:
# Function to evaluate a genome's fitness
def evaluate(genome):
    genome.fitness = 1.0 / (1.0 + abs(math.cos(genome.genes) - math.sin(genome.genes)))

winner = evolution(1, 1, 1000)
print("Champion -- Genes = " + str(winner.genes) + " Fitness = " + str(winner.fitness))

#### 3.3.2. Growing the Population

The performance of evolution can heavily depend on chosen parameters.  These parameters control the balance between exploration (finding new solutions), exploitation (optimizing existing solution), and how we traverse the fitness landscape.  The first paremeters we will look at are the <i>population</i> parameters.  These parameters include <b>$\mu$</b> (the population size) and <b>$\lambda$</b> (the offspring size).

#### Population Size ($\mu$)

Looking at the first parameter $\mu$, this represents the number of individuals that will be maintained in the population.  One way of looking at it is $\mu$ is the number of parallel search paths we are actively searching through, that is, each individual represents a particular search area in the landscape.  In this way, $\mu$ controls the degree of exploration in the algorithm.  The higher the value, the more areas that are being searched.  However, the trade-off is that the greater the number of areas being searched, the less each area will have time be optimized.  Examine the results below as $\mu$ varies.  Take particular note of how quickly the average fitness converges on the best fitness, which is a measure of how exploitative the algorithm is.

In [ ]:
# Function that will perform evolution 
# with the given parameters
def interactive_iteration(generations=1000,
                          mu=1):
    if(mu == 0):
        mu = 1
    winner = evolution(mu, 1, generations)

    print("Champion -- Genes = " + str(winner.genes) + " Fitness = " + str(winner.fitness))
    
# Create interactive sliders for parameters
interact(interactive_iteration, generations=(0,10000,10),
                                mu=(0,1000, 10))

#### Offspring Size ($\lambda$)

The second parameter, $\lambda$, similarly balances exploration versus exploitation.  The greater the number of offspring generated, the more search is conducted around the existing population points.  For example, with $\mu = 1$ and $\lambda = 100$, 100 different points around the existing indivudal are explored.  The exploitation tradeoff with $\lambda$ is that the increased number of offspring correspond the increased computational cost in the number of evaluations per generation, thus for the same computational cost you can perform fewer generations (less optimization). Examine the results below as $\lambda$ varies.

In [ ]:
# Function that will perform evolution 
# with the given parameters
def interactive_iteration(generations=1000,
                          mu=1,
                          lamb=1):
    if(mu == 0):
        mu = 1
    if(lamb == 0):
        lamb = 1
    winner = evolution(mu, lamb, generations)

    print("Champion -- Genes = " + str(winner.genes) + " Fitness = " + str(winner.fitness))
    
# Create interactive sliders for parameters
interact(interactive_iteration, generations=(0,10000,10),
                                mu=(0,1000, 10),
                                lamb=(0,1000, 10))

#### Exercise: Explore the interaction between $\mu$ and $\lambda$

Now, try out different combinations of $\mu$ and $\lambda$ on your own, with different fitness functions.

In [ ]:
def evaluate(genome):
    genome.fitness = math.cos(genome.genes + 3) / (1.0 + abs(genome.genes))
    
# Create interactive sliders for parameters
interact(interactive_iteration, generations=(0,10000,10),
                                mu=(0,1000, 10),
                                lamb=(0,1000, 10))

In this section, we examined the effect that population and offspring size can have in evolution.  In the next section, we will examine the importance of selection in evolution.

#### 3.3.3. Selecting the best!

<i>Selection</i> plays a significant role in how well the evolutionary algorithm performs and there are a number of different approaches to selection.  Additionally, there are two points where selections comes into effect: Selecting parents to produce children, and, Selecting members of the current population and offspring to become part of the next population.  By convention, selective pressure is often applied at only one of these points, leaving the other to uniform random (neutral) selection. In this section we will discuss different selection approaches and what selection <i>means</i> at the different points.

#### The Pool of Candidates

First, let us examine selecting for the next population.  Selecting for the next population has meaning in the sense of asking, of the given points in the space we are currently at, which points do we want to maintain in our seach.  Looking at our previously defined select_population function:

In [ ]:
# Select a new population of individuals from the current and offspring
def select_population(currentPopulation, offspring): 
    
    # Get the size of the current population
    popSize = len(currentPopulation)
    
    # Initialize an empty list for the new population
    newPopulation = [] 
    
    # Combine the current and offspring populations
    currentPopulation.extend(offspring) 
    
    # Get the combined size of current and offspring populations
    combinedSize = len(currentPopulation)
    
    # Until the population for the next generation
    # is the size of the current population
    # keep selecting genomes
    while(len(newPopulation) < popSize): 
        
        # Select an index for the first candidate genome
        candidateOneIdx = random.randint(combinedSize)  
        # Select an index for the second candidate genome
        candidateTwoIdx = random.randint(combinedSize)  
        
        # If the same index is selected
        if(candidateOneIdx == candidateTwoIdx): 
            # Increment second index by one and make sure it is within bounds
            candidateTwoIdx = (candidateTwoIdx + 1)%combinedSize
            
        # Set the selected genome to the first candidate
        selected = currentPopulation[candidateOneIdx]  
        
        # If the second candidate's fitness is better
        # set it to be selected
        if(currentPopulation[candidateTwoIdx].fitness > selected.fitness): 
            selected = currentPopulation[candidateTwoIdx]
            
        # Add the selected genome to the new population
        # and remove it from the pool of candidates
        newPopulation.append(selected) 
        currentPopulation.remove(selected)
        
        # Decrement the pool count
        combinedSize -= 1
        
    # Return the new population
    return newPopulation 

In this function, there are two items that define this selection approach.  First, the current population and the offspring are combined into a single pool for selection.  This pooling means that offspring compete with parents for slots in the next population, referred to as ($\mu + \lambda$), or, inter-generational selection.  Alternatively, generational selection, or ($\mu$,$\lambda$), eliminates parents to make space for the offspring and then competition for slots only exists among the offspring.  As an exercise, try to modify the above function, such that ($\mu$,$\lambda$) selection is taking place.  Hint: There are two conditions to consider (1) $\mu > \lambda$, and, (2) $\mu \leq \lambda$.  Under the first condition, members of the population must be eliminated until there is $\lambda$ space, and the second condition means that the offspring must be reduced to $\mu$ members.

In [ ]:
# Create interactive sliders for parameters
interact(interactive_iteration, generations=(0,10000,10),
                                mu=(0,1000, 10),
                                lamb=(0,1000, 10))

#### Methods of Selection

The second important part of this function is <i>how</i> selection is performed.  In this case, we can see that two genomes are selected (candidateOneIdx and candidateTwoIdx) at random from the candidate pool.  The fitness of these two candidates are then compared and the fitter of the two are selected.  This selection process is called <i>Tournament Selection</i>.  In this case, it is a tournament of size two, noted as $k=2$.  The $k$ value can be take on any value less than or equal to the size of the candidate pool; however, the larger the $k$, the greater the selective pressure.  Tournament selection is often chosen because it is computationally easy and because different $k$ probabilistically approximate most other selection approaches.

Other selections approaches include (listed here from weakest to strongest pressure):
<ul>
<li><b>Uniform</b>: Each candidate has equal probability of being selected (fitness is not compared)</li>
<li><b>Fitness Proportional</b>: Each candidate is assigned a probability that is proportional to its fitness relative to the population</li>
<li><b>Linear Ranking</b>: Each candidate is assigned a linearly decreasing probability relative to its ranking order in the population</li>
<li><b>Non-linear Ranking</b>: Each candidate is assigned a non-linearly decreasing probability relative to its ranking order in the population</li>
<li><b>Truncation</b>: The top $n$ candidates have probability $1/n$, the rest are discarded</li>
</ul>

Below, the select_population function has been modified to do a tournament of size 3.  As an exercise, attempt to implement a tournament selection of arbitrary size or one of the above non-tournament selection approaches.

In [ ]:
# Select a new population of individuals from the current and offspring
def select_population(currentPopulation, offspring): 
    
    # Get the size of the current population
    popSize = len(currentPopulation) 
    
    # Initialize an empty list for the new population
    newPopulation = [] 
    
    # Combine the current and offspring populations
    currentPopulation.extend(offspring) 
    
    # Get the combined size of current and offspring populations
    combinedSize = len(currentPopulation)
    
    # Until we have selected to the population size
    while(len(newPopulation) < popSize): 
        
        # Select indicies for the candidate genomes
        candidateOneIdx = random.randint(combinedSize)  
        candidateTwoIdx = random.randint(combinedSize)  
        candidateThreeIdx = random.randint(combinedSize)
        
        # If all three indicies are the same
        if(candidateOneIdx == candidateTwoIdx and candidateOneIdx == candidateThreeIdx):
            
            # Increment second index by one,
            # make sure it is within bounds
            # and increment the third index
            # by two
            candidateTwoIdx = (candidateTwoIdx + 1)%combinedSize 
            candidateThreeIdx = (candidateThreeIdx + 2)%combinedSize
        
        # If first and second index are the same
        if(candidateOneIdx == candidateTwoIdx): 
            # Increment second index by one
            candidateTwoIdx = (candidateTwoIdx + 1)%combinedSize
            
        # If first and third index are the same
        if(candidateOneIdx == candidateThreeIdx): 
            # Increment third index by one
            candidateThreeIdx = (candidateThreeIdx + 1)%combinedSize
            
        # If second and third index are the same
        if(candidateTwoIdx == candidateThreeIdx): 
            # Increment third index by one
            candidateThreeIdx = (candidateThreeIdx + 1)%combinedSize   
        
        # Set the selected genome to the first candidate
        selected = currentPopulation[candidateOneIdx]  
        
        # If the second candidate's fitness is better
        # Set selected to the second candidate
        if(currentPopulation[candidateTwoIdx].fitness > selected.fitness): 
            selected = currentPopulation[candidateTwoIdx] 
        
        # If the third candidate's fitness is better
        # Set selected to the third candidate
        if(currentPopulation[candidateThreeIdx].fitness > selected.fitness): 
            selected = currentPopulation[candidateThreeIdx]
            
        # Add the selected genome to the new population
        newPopulation.append(selected) 
        # Remove the selected genome from the current population
        currentPopulation.remove(selected)
        # Decrement the size of the current population by one
        combinedSize -= 1 
        
    # Return the new population    
    return newPopulation 

In [ ]:
# Create interactive sliders for parameters
interact(interactive_iteration, generations=(0,10000,10),
                                mu=(0,1000, 10),
                                lamb=(0,1000, 10))

#### Selecting Parents

Alternatively, selection pressure can be applied through the selection of parents.  This point of selection has a subtly different meaning than selecting for the next population and indeed different selection <i>methods</i> have different meanings here.  When selecting parents, what the pressure is telling evolution is which points do we currently want to explore around.  Examine the previsouly defined select_parents function below: 

In [ ]:
# Selects the given number of parents from the given population
def select_parents(population, number): 
    
    # Initialize an empty list of parents
    parents = []  
    
    # Get the number of individuals in the given population
    populationSize = len(population)
    
    # For the number of parents
    for x in range(number): 
        
        # Select a random genome from the population
        # with uniform probability to be a parent
        parents.append(population[random.randint(populationSize)]) 
    
    # Return the parents list
    return parents 

First, note that there is no selection pressure here, that is, the selection is uniform random.  Second, replacement is taking place, thus any genome may occur multiple times in the parent list thereby producing multiple children.  This replacement is where the subtly different meaning impacts the different selection methods.  For example, fitness proportional selection means the proportion of new search points around a genome is determined by its relative fitness, that is, the selection probability assigned to a genome represents its expected number of children. As an example, below we remove the pressure from select_population, and add it to select_parents by making population selection uniform random and the parent selection done through tournament selection.

In [ ]:
# Selects the given number of parents from the given population
def select_parents(population, number):
    
    # Initialize an empty list of parents
    parents = []  
    # Get the number of individuals in the given population
    populationSize = len(population) 
    
    # For the number of parents
    for x in range(number): 
        
        # Select indicies for the candidate parents
        candidateOneIdx = random.randint(populationSize)  
        candidateTwoIdx = random.randint(populationSize)  
        
        # Set selected to the first choice
        selected = population[candidateOneIdx]
        
        # If second choice has better fitness
        # Set selected to the second choice
        if(selected.fitness < population[candidateTwoIdx].fitness): 
            selected = population[candidateTwoIdx]
            
        # Add the selected parent
        parents.append(selected) 
        
    # Return the parents list
    return parents 

# Select a new population of individuals from the current and offspring
def select_population(currentPopulation, offspring):
    
    # Get the size of the current population
    popSize = len(currentPopulation)
    # Initialize an empty list for the new population
    newPopulation = [] 
    
    # Combine the current and offspring populations
    currentPopulation.extend(offspring) 
    # Get the combined size of current and offspring populations
    combinedSize = len(currentPopulation) 
    
    # Until we reach the correct population size
    while(len(newPopulation) < popSize):
        
        # Select index for candidate
        candidateOneIdx = random.randint(combinedSize)
        
        # Get the selected genome
        selected = currentPopulation[candidateOneIdx]
        
        # Add the selected genome to the new population
        newPopulation.append(selected) 
        
        # Remove the selected genome from the current population
        currentPopulation.remove(selected)
        
        # Decrement the size of the candidate pool by one
        combinedSize -= 1 
        
    # Return the new population    
    return newPopulation 

In [ ]:
# Create interactive sliders for parameters
interact(interactive_iteration, generations=(0,10000,10),
                                mu=(0,1000, 10),
                                lamb=(0,1000, 10))

In this section, we examined the different selection approaches and portions of evolution where selection occurs.  In the next section, we will see how evolutionary algorithms produce new solutions.

#### 3.3.4. Reproduction: Where baby solutions come from...

How new solutions are created plays a pivotal role in the performance of evolutionary algorithms.  They define what points in the search space are reachable, how quickly you can traverse the search space, when you converge to a solution, and to where the solutions converges. As mentioned earlier there are two primary means of creating new solutions in evolution: (1) <i>Mutation</i>, and, (2) <i>Recombination</i>.  We shall examine these two concepts in the following sections.  However, first we will modify our evaluator and genome to deal with multiple real-valued numbers as genes in order to demonstrate variations in muation and recombination.

In [ ]:
# Creates a new genome with two
# real-valued numbers are genes
# with a range of [-3,3]
def new_genome():
    
    # Initialize genes to an empty list
    genes = []  
    
    # Add one value between -3 and 3 as the first gene
    genes.append(random.uniform(-3, 3)) 
    # Add a second value between -3 and 3 as the second gene
    genes.append(random.uniform(-3, 3)) 
    
    # Create a new genome
    return Genome(-10000, genes)  

# Evaluation of the genome is to
# maximize a function of two variables
def evaluate(genome):
    genome.fitness = -math.log(abs((100*(genome.genes[0]**2 - genome.genes[1])**2 + (1 - genome.genes[0])**2)) + 1)

#### Mutation: Copying with error

<i>Mutation</i> is the act of changing the genes of a genome to produce a new genome that is slightly different from the original.  In evolutionary algorithms, this is (mostly) part of <i>Asexual Reproduction</i>, that is, creating a exact clone of a parent genome and then applying mutations to the clone to produce a variant of the parent. To illustrate, we will implement a mutation function for our above multi-gene genome.

In [ ]:
# Mutates a multiple real-value
# genome
def mutate_genome(genome):
    
    # Select one of the real-values uniform randomly
    geneToMutateIdx = random.randint(len(genome.genes))
    
    # Perturbs that value by adding random value in [-0.01, 0.01]
    genome.genes[geneToMutateIdx] += random.uniform(-0.01, 0.01)

Looking at the above function, it takes in an existing genome (clone of a parent), selects a single gene from among an array of real-valued numbers, and then adds a value between -0.01 and 0.01 to it. Let us look at how this basic mutation performs on the task.

In [ ]:
# Create interactive sliders for parameters
interact(interactive_iteration, generations=(0,10000,10),
                                mu=(0,1000, 10),
                                lamb=(0,1000, 10))

#### Mutation Power

In the case of real-valued numbers, one means of varying how mutation works is by changing the power, or, amount by which the genes are perturbed.  Below, see the effect of turning up the power of the mutations.

In [ ]:
# Mutates a multiple real-value
# genome
def mutate_genome(genome):
    
    # Select one of the real-values uniform randomly
    geneToMutateIdx = random.randint(len(genome.genes))
    
    # Perturbs that value by adding random value in [-10, 10]
    genome.genes[geneToMutateIdx] += random.uniform(-10, 10)

In [ ]:
# Create interactive sliders for parameters
interact(interactive_iteration, generations=(0,10000,10),
                                mu=(0,1000, 10),
                                lamb=(0,1000, 10))

As can be seen above, changing the power of the mutation by switching from a a muation that adds a value in ($-0.01, 0.01$) to a value in ($-10, 10$) significantly alters the algorithms behavior.  Explore how different mutations power can have different impacts by changing the value of <i>power</i> below.

In [ ]:
power = 0.1

# Mutates a multiple real-value
# genome
def mutate_genome(genome):
    
    # Select one of the real-values uniform randomly
    geneToMutateIdx = random.randint(len(genome.genes))
    
    # Perturbs that value by adding random value in [-power, power]
    genome.genes[geneToMutateIdx] += random.uniform(-power, power)

In [ ]:
# Function that will perform evolution 
# with the given parameters
def interactive_iteration(generations=1000,
                          mu=1,
                          lamb=1,
                          mutation_power=1.0):
    if(mu == 0):
        mu = 1
    if(lamb == 0):
        lamb = 1
    global power
    power = mutation_power
    winner = evolution(mu, lamb, generations)

    print("Champion -- Genes = " + str(winner.genes) + " Fitness = " + str(winner.fitness))
    
# Create interactive sliders for parameters
interact(interactive_iteration, generations=(0,10000,10),
                                mu=(0,1000, 10),
                                lamb=(0,1000, 10),
                                mutation_power=(0, 10, 0.05))

#### Distribution of Mutations

Related to mutation power, another means of changing mutation is to alter the distribution of mutations selected, that is, we can alter the above function to favor smaller mutations rather than larger mutations.  The above function selects mutations from a uniform distribution, however we could also choose mutations from a normal distribution.  Experiment with the normal distribution below by varying sigma (variance).

In [ ]:
sigma = 0.1

# Mutates a multiple real-value
# genome
def mutate_genome(genome):
    
    # Select one of the real-values uniform randomly
    geneToMutateIdx = random.randint(len(genome.genes))
    
    # Perturbs that value by adding random value in [-power, power]
    genome.genes[geneToMutateIdx] += random.normal(sigma)

In [ ]:
# Function that will perform evolution 
# with the given parameters
def interactive_iteration(generations=1000,
                          mu=1,
                          lamb=1,
                          mutation_power=1.0):
    if(mu == 0):
        mu = 1
    if(lamb == 0):
        lamb = 1
    global power
    power = mutation_power
    global sigma
    sigma = mutation_power
    winner = evolution(mu, lamb, generations)

    print("Champion -- Genes = " + str(winner.genes) + " Fitness = " + str(winner.fitness))
    
# Create interactive sliders for parameters
interact(interactive_iteration, generations=(0,10000,10),
                                mu=(0,1000, 10),
                                lamb=(0,1000, 10),
                                mutation_power=(0, 10, 0.05))

#### Mutation Rate

Finally, another important consideration for mutations is the number of genes mutated per mutation event.  The above mutation functions each select a single gene, at random, and then mutate it.  However, there may be cases where it is advantageous to allow multiple genes to be mutated in a single mutation event.  For example, if you have values $x$ and $y$ into a function $f$, such that $f(x,y + \Delta y) < f(x,y)$ and $f(x + \Delta x,y) < f(x,y)$, but $f(x + \Delta x,y + \Delta y) > f(x,y)$.  In this case, the only path to optimize the function involves moving $x$ and $y$ at the same time. Thus many approaches to evolutionary computation with multiple genes implement the mutation function such that each gene has a probability of being selected, thereby setting an expected number of genes mutated through this probability.  Below, such an approach is implemented.  Examine the effects as the probability (mutation rate) is raised and lowered by manipulating pMutateGene.

In [ ]:
pMutateGene = 0.75

def mutate_genome(genome):
    # For each gene
    for x in range(len(genome.genes)): 
        # Randomly decide if the gene is to be mutated
        if(random.uniform(0, 1) < pMutateGene): 
            # Mutate this gene
            genome.genes[x] += random.normal(0, sigma) 

In [ ]:
# Function that will perform evolution 
# with the given parameters
def interactive_iteration(generations=1000,
                          mu=10,
                          lamb=10,
                          mutation_power=1.0,
                          prob_mutate_gene=0.75):
    if(mu == 0):
        mu = 1
    if(lamb == 0):
        lamb = 1
    global power
    power = mutation_power
    global sigma
    sigma = mutation_power
    global pMutateGene
    pMutateGene = prob_mutate_gene
    winner = evolution(mu, lamb, generations)

    print("Champion -- Genes = " + str(winner.genes) + " Fitness = " + str(winner.fitness))
    
# Create interactive sliders for parameters
interact(interactive_iteration, generations=(0,10000,10),
                                mu=(0,1000, 10),
                                lamb=(0,1000, 10),
                                mutation_power=(0, 10, 0.05),
                                prob_mutate_gene=(0,1, 0.05))

Mutations play a very important role in evolutionary algorithms and this section just scratched the surface in the breadth and depth of mutation approaches.  The next section will examine the topic of Recombination.

#### 3.3.5. Recombination: Playing Legos with genes

As mentioned earlier, evolutionary computation is inspired by the natural process of evolution.  In nature, one method of inducing variation is mutation, as described above. Another approach in nature is for two or more existing genomes to be recombined by taking portions of each to create a new genome.  In this way, promising pieces of each solution can be brought together to make a new, hopefully better, solution.  To this end, we change the reproduce function to create new genomes through recombination and define a recombine function that will take two genomes and return a new genome that is a recombination of the given two.

In [ ]:
# Creates a new genome by recombining
# the genes of the two given parents
def recombine(parent1, parent2):
    
    # Clone the first parent
    child = parent1.clone()  
    
    # Set the second gene of the child to the value 
    # of the second gene of the second parent
    child.genes[1] = parent2.genes[1] 
    
    # Return the child
    return child 
    
# Takes in a set of parents and produces an equivalent number of offspring
def reproduce(parents): 
 
    # Initialize an empty list of offspring
    offspring = []
    
    # For each parent, create offspring through sexual reproduction
    for parent in parents: 

        # Select a second parent
        parent2 = parents[random.randint(len(parents))]
        
        # Create a child by recombining the genes from parent and parent2
        child = recombine(parent, parent2)
        
        # Set its fitness to a very low value
        child.fitness = -10000 
        
        # Add the child to the offspring list
        offspring.append(child) 
        
    # Return the list of offspring
    return offspring 

Note that through pure recombination, no new genes are introduced.  Instead, whatever we initialize the population with is what we have and evolution can only optimize to the extent that the initial values allow.  Thus the population and offspring size become even more important when dealing with recombination only reproduction.  Explore the effect that switching to recombination has below.  Feel free to explore different evaluation functions!

In [ ]:
def evaluate(genome):
    genome.fitness = -math.log(abs((100*(genome.genes[0]**2 - genome.genes[1])**2 + (1 - genome.genes[0])**2)) + 1)
      
# Create interactive sliders for parameters
interact(interactive_iteration, generations=(0,10000,10),
                                mu=(0,1000, 10),
                                lamb=(0,1000, 10),
                                mutation_power=(0, 10, 0.05),
                                prob_mutate_gene=(0,1, 0.05))

#### Mutation AND Recombination

Finally, let us bring it all together by combining <i>mutation</i> and <i>recombination</i>.  Below, we implement a a reproduce method that allows evolution to either produce solutions through mutation or recombination, control by a probability: pRecombination.

In [ ]:
# Probability of performing
# recombination rather than mutation
pRecombination = 0.5

# Takes in a set of parents and produces 
# an equivalent number of offspring
def reproduce(parents):
    
    # Initialize an empty list of offspring
    offspring = [] 
    
    # For each parent, create offspring 
    for parent in parents: 
        child = None
        
        # If we reproduce through cloning and mutation
        if(random.uniform(0, 1) > pRecombination):
            # Clone the parent
            child = parent.clone();
            # Mutate the clone
            mutate_genome(child)
        # Else we reproduce through recombination
        else:
            # Select a second parent
            parent2 = parents[random.randint(len(parents))]
            # Create a child by recombining the genes from parent and parent2
            child = recombine(parent, parent2) 
            
        # Set child's fitness to a very low value
        child.fitness = -10000 
        # Add the child to the offspring list
        offspring.append(child) 
    
    # Return the list of offspring
    return offspring 

Now we have a simple algorithm that implements the primary features of evolutionary approaches.  Experiment with different parameter settings and evaluation functions!  We will explore advanced topics in the next section.

In [ ]:
def evaluate(genome):
    genome.fitness = -math.log(abs((100*(genome.genes[0]**2 - genome.genes[1])**2 + (1 - genome.genes[0])**2)) + 1)

    # Function that will perform evolution 
# with the given parameters
def interactive_iteration(generations=1000,
                          mu=10,
                          lamb=10,
                          mutation_power=1.0,
                          prob_mutate_gene=0.75,
                          prob_recombine=0.5):
    if(mu == 0):
        mu = 1
    if(lamb == 0):
        lamb = 1
    global power
    power = mutation_power
    global sigma
    sigma = mutation_power
    global pMutateGene
    pMutateGene = prob_mutate_gene
    global pRecombine
    pRecombine = prob_recombine
    winner = evolution(mu, lamb, generations)

    print("Champion -- Genes = " + str(winner.genes) + " Fitness = " + str(winner.fitness))
    
# Create interactive sliders for parameters
interact(interactive_iteration, generations=(0,10000,10),
                                mu=(0,1000, 10),
                                lamb=(0,1000, 10),
                                mutation_power=(0, 10, 0.05),
                                prob_mutate_gene=(0,1, 0.05),
                                prob_recombine=(0,1,0.05))

### 3.4. (Optional) Advanced Topics: Theory, How to make policies, Co-Evolution, and Prison

In this section, we are briefly going to cover basic theory of how evolutionary algorithms work and then discuss how evolution can search through policies and the idea of co-evolution through the Prisoner's Dilemma domain.

#### 3.4.1. Building blocks and Schemas

To examine some basic theory underlying evolutionary algorithms, we are going to implement a genetic algorithm type approach (i.e. evolving strings of binary valued bits). 

In [ ]:
# Number of bits in the genome
numBits = 32 
# Probability of mutating a bit
pMutateGene = 1.0/numBits
# Probability of recombining two genomes
pRecombination = 0.5 
# Population Size
mu = 10 
# Offspring size
lamb = 10 

# Creates and returns a new random genome
def new_genome(): 
    bits = []
    for x in range(numBits):
        bits.append(random.randint(2))
    # A random genome for this domain is array of 0's and 1's
    return Genome(-10000, bits) 

# Mutation takes a genome and alters 
# its genes to create a new solution instance
def mutate_genome(genome): 
    # For the number genes
    for x in range(len(genome.genes)): 
        # If randomly selected number is 
        # below probability of mutating single gene
        if(random.uniform(0, 1) < pMutateGene): 
            # Mutate this gene by flipping 0 to 1, or, 0 to 1
            genome.genes[x] = (genome.genes[x] + 1) % 2 

# Creates offspring by recombining
# the genes of the given parents
def recombine(parent1, parent2):  
    # Clone parent 1
    child = parent1.clone()
    # Choose a point where the child's genes 
    # start to come from parent 2
    crossoverPoint = random.randint(1, len(child.genes)-1) 
    # Set the child genes to parent 1's genes
    # up to the crossover point
    child.genes = child.genes[:crossoverPoint]
    # Complete the child's genes with parent 2's
    # genes after the crossover point
    child.genes.extend(parent2.genes[crossoverPoint:])
    # Return the child
    return child 

# Evaluator that is the OneMax Domain
def evaluate(genome):
    genome.fitness = 0
    # Implementation of OneMax domain 
    # (i.e. maximize the number of 1 bits in the genome)
    # Sums the number of bits set to 1
    for x in genome.genes: 
        genome.fitness += x

winner = evolution(mu, lamb, 1000)
print("Champion -- Genes = " + str(winner.genes) + " Fitness = " + str(winner.fitness))

The above code implements the OneMax domain for genetic algorithms.  This is a very simple domain that provides a sanity check to make sure your algorithm is working and a platform for exploring properties of evolutionary algorithms.  In this example, the genome consists of an array of bits, that is, an array of values that are either 0 or 1.  The evaluator determines fitness by counting the number of array values that are set to 1.  Thus the optimal array is filled with 1's. Note: The recombine method above performs <i>single-point crossover</i> (i.e. the recombination is performed by splitting the parent arrays at a single point).  As an exercise, try implementing multi-point crossover, wherein recombination splits the genes array at multiple points to create a child.

The OneMax domain allows us to explain the concept of 'building blocks'.  A hypothetical basis for the performance of evolution is building blocks, that is, beneficial genes will begin to occur together and form into modules.  These beneficial groupings will then propagate throughout the system and combine with other beneficial groupings to make even better solutions.  This 'building blocks' hypothesis was formally defined by John Holland (1975) in the Schema Theorem for genetic algorithms.

A schema is template of a bit string consisting of <i>defined</i> bits (0, 1), or <i>wild cards</i> (\*).  Thus a particular schema may be $H = 1***1$, meaning the schema is 5 bits, beginning and ending with 1 and the rest we don't care.  The idea is that such schemas represent building blocks throughout a genetic algorithm.  If a particular schema on average contributes to more fit genomes, they will then begin occuring more frequently in the system, that is, schemas are <i>implicitly</i> evaluated through the evolutionary process.  Because each genome will represent multiple schema at any given time, this gives rise to the idea of <i>implicit parallelism</i>, wherein many schemas are being evaluated simultaneously.

This is the extent to which we will cover the theoretical foundations, however such theories allow mathematical analysis of the effect that selection, recombination, and mutation have through the effect such actions have on schema.  The next section we will explore how evolution can create agent policies and the idea of co-evolution through game theory.

#### 3.4.2. The Prisoner's Dilemma

To illustrate learning policies, we introduce the well known Prisoner's Dilemma problem from Game Theory.  In the Prisoner's Dilemma, two criminal conspiritors are caught by the police and imprisoned.  The prisoners are separated into two different interrogation rooms.  Now, the police do not have enough evidence to convict them on the full charges of their crimes, so they need confessions.  To each prisoner the police offer the same deal: Rat out your partner and we'll give you a lighter sentence.  If neither defects on their partner, they both serve a short prison sentence.  If only one defects, the defector gets the minimum sentence, while the one who remains silent receives a very long sentence.  Finally, if both defect, they both get long sentences.  Thus defecting, which is the path to the minimum sentence, can result in a worse outcome than if both stayed silent, thereby creating the dilemna.  Can the prisoners trust each other to not go for the minimum sentence? 

Formally, you have two agents that can either cooperate (C), or defect (D), and the reward structure for an individual agent is such that DC > CC > DD > CD, where the first letter the agent's action and the second letter is the co-agent's action.  Such a reward structure can be envisioned by the following matrix: 
\begin{array}{ccc}
 & C & D \\
C & 3,3 & 0,5 \\
D & 5,0 & 1,1 \end{array}
A more interesting extension, called the Iterated Prisoner's Dilemna, extends the game to multiple moves, that is, the agents have to play the game several times in a row and the rewards accumulate across all the decisions and the agents are aware of the results of the previous games.  Thus if your opponent always defects, you can respond in kind, rather than always losing.  

A challenge for learning policies to such competitive domains is evaluation.  For example, if we wanted to learn policies to play chess, we would need an already built system that can play chess... but if we have such a system, why do we need to learn one?  Additionally, if the competitor that is evaluated against is too good, all policies will initially perform equally poorly against it making learning difficult.  Finally, the policies that are learned will be optimized to play against that particular opponent rather than an arbitrary opponent. Addressing these concerns is the field of <i>Co-evolution</i>.  In co-evolution, instead of an oracle evaluator that tells us the exact fitness of each individual, individuals compete (or cooperate) together to determine their fitness.  Thus, going back to the chess example, genomes within the population would compete against each other in chess games to determine how well they perform.  The hope would there would be an 'arms race' or Red Queen effect, wherein genomes must continually improve in performance to maintain fitness relative to the other individuals in the population. Below we modify our evolutionary approach to implement co-evolution to learn policies for the Iterated Prisoner's Dilemna.

In [ ]:
def evolution(populationSize, offspringSize, maxGenerations):
    
    currentGeneration = 0 # Current generation being executed
    
    avgFitnessByGeneration = [] # Array to hold generational average fitness values
    bestFitnessByGeneration = [] # Array to hold generational champion fitness values
    championValuesByGeneration = [] # Array to hold generational champion values
    
    population = []  # Initialize an empty population
    bestGenome = None # Set the best genome to a null value
    
    avg = 0.0  # Initialize variable for calculating population average fitness to 0
    for x in range(populationSize): # For the size of the population
        population.append(new_genome()) # Createa new random genome and add it to the population
    for x in range(populationSize): # For the size of the population
        evaluate(population[x], population) # Evaluate each genome against the population
        avg += population[x].fitness # Add its fitness score to the average
        if(bestGenome is None or population[x].fitness > bestGenome.fitness):  # If the best genome hasn't been set, or, this
                                                                               # genome has better fitness
            bestGenome = population[x] # Set the bestGenome to this genome
    
    # Add the metrics to their various arrays for tracking
    avgFitnessByGeneration.append(avg/populationSize)
    bestFitnessByGeneration.append(bestGenome.fitness)
    championValuesByGeneration.append(bestGenome.genes)
    
    while(currentGeneration < maxGenerations): # For the maximum number of generations
        
        parents = select_parents(population, offspringSize) # Select a number of parents equal to the number of offspring to create
        offspring = reproduce(parents) # Create a number of offspring by reproducing the selected parents
        bestGenome = None
        for child in offspring: # For each of the offspring
            evaluate(child, population) # Evaluate the offspring against the existing population
            if(bestGenome is None or child.fitness > bestGenome.fitness): # Update the best genome is applicable
                bestGenome = child
        
        population = select_population(population, offspring) # Select a new population from the current population and offspring
        
        avg = 0.0 # Init average to 0
        for genome in population: # For each genome in the population
            avg += genome.fitness # Add its fitness to the average
   
        # Add the metrics to the various arrays
        avgFitnessByGeneration.append(avg/populationSize)
        bestFitnessByGeneration.append(bestGenome.fitness)
        championValuesByGeneration.append(bestGenome.genes)
        
        currentGeneration += 1 # Increment current generation by one
    
    # Plot the best fitness, average fitness, and the champion gene values by generation
    labels = range(len(bestFitnessByGeneration))
    subplot(2,1,1)
    plot(labels, bestFitnessByGeneration, labels, avgFitnessByGeneration)
    ylabel("Fitness")
    legend(["Best", "Avg."], loc='lower center', bbox_to_anchor=(0.5, 1.05),
          ncol=3, fancybox=True, shadow=True)
    subplot(2,1,2)
    plot(labels, championValuesByGeneration)
    ylabel("Gene Values")
    
    
    return bestGenome # Return the best genome

In [ ]:
numBits = 5  # Five bits to represent the prisoner's policy, with meaning defect, and 0 meaning cooperate
             # The first bit represents an action to take with no history, followed by history of CC, DC, CD, and DD
rewards = [3, 5, 0, 1] # Rewards for CC, DC, CD, DD
gameIterations = 100

def dilemma(policy, opponent, iterations): # Score a given policy against an opponent for a number of iterations
    score = 0.0 # Initial score of 0
    lastOutcome = 0 # Last outcome with (0 = No history, 1 = CC, 2 = DC, 3 = CD, 4 = DD)
    for x in range(iterations): # For the number of iterations
        policyDecision = policy[lastOutcome] # Get the policy's decision given last outcome
        opponentDecision = opponent[lastOutcome] # Get opponent's decision given last outcome
        lastOutcome = 1 + policyDecision * 1 + opponentDecision * 2 # Set the last outcome
        score += rewards[lastOutcome - 1] # Get the score given the outcome
    return score / iterations # Dive the score by number of iterations

def evaluate(genome, opponents):
    genome.fitness = 0
    for x in opponents: # For each opponent, evaluate the genome against it
        genome.fitness += dilemma(genome.genes, x.genes, gameIterations)
    genome.fitness = genome.fitness / len(opponents) # Divide fitness by number of opponents for an average    

Several modifications can be seen in the above code.  For the evolutionary loop, we now pass in a set of opponents to evaluate a particular genome against for the evaluate function.  The evaluate function now takes a genome and a set of opponents and evaluates that genomes policy against each of these opponents and averages the performance.  The dilemma functions performds the Iterated Prisoner's Dilemma between a given policy and an opponent for a number of iterations.  Experiment below with different parameters and observe the significant difference that can occur in a co-evolutionary fitness scores.

In [ ]:
mu = 100 # Population Size
lamb = 10 # Offspring size
totalGenerations = 1000 # Number of generations of evolution 
pMutateGene = 1.0/numBits # Probability of flipping a bit
pRecombination = 0 # Probability of recombining genomes rather than mutating
rewards = [3, 5, 0, 1] # Rewards for CC, DC, CD, DD
gameIterations = 100 # Number of iterations for the Prisoner's Dilemma

winner = evolution(mu, lamb, totalGenerations)
print("Champion -- Genes = " + str(winner.genes) + " Fitness = " + str(winner.fitness))

### 3.5. Evolutionary Computation Conclusion

This ends our introduction to evolutionary computation.  We completed an overview of a basic evolutionary algorithm, demonstrated the key components of the algorithm and the effects they have, and discussed soem preliminaries on theory and applications to policy search for reinforcement learning.  As a reminder, the key parts to implementing the evolutionary algorithms are:

<ul>
<li><b>Genomes:</b> Representations of problem solutions</li>
<li><b>Evaluation:</b> Means of determining genome's fitness</li>
<li><b>Reproduction:</b> Process by which existing genomes (<b>parents</b>) produce new genomes (<b>offspring, or children</b>), through <b>mutation</b> or <b>recombination</b></li>
<li><b>Selection:</b> A process by which either (1) parents are chosen to create offspring, or, (2) individuals are chosen to die and be replaced </li>
</ul>

There is a vast amount of literature out there if you wish to learn more, from basic text books like <i>Evolutionary Computation: a unified approach</i> by Kenneth De Jong and <i>An Introduction to Genetic Algorithms</i> by Melanie Mitchell, to cutting edge international conferences, such as ACM's Genetic and Evolutionary Computation Conference (GECCO) and IEEE's Congress on Evolutionary Computation (CEC).  

Evolutionary computation has (pardon the pun) evolved over time from its simple optimization roots, to increasingly complex areas, such as swarm intelligence and artificial life.  There remains debate whether evolutionary algorithms are reinforcement learning; however, there is little debate that problems that reinforcement learning address can often also be solved by evolutionary approaches.